In [50]:
import os
import shutil
import re
import numpy as np
from collections import defaultdict
from operator import itemgetter

In [10]:
filenames = ['./models/part-00000.txt', './models/part-00001.txt']

In [11]:
word_search_re = re.compile(r"[\w']+")

In [12]:
def load_model(filenames):
    """ parse files with models for naive bayes
    """
    model = defaultdict(lambda: defaultdict(float))
    for filename in filenames:
        with open(filename) as inf:
            for line in inf:
                word, values = line.split(maxsplit=1)
                word = eval(word)
                values = eval(values)
                model[word] = values
    return model

In [13]:
blogposts_model = load_model(filenames)

In [43]:
# check the probabilities for each class
blogposts_model['yes']

{'female': 0.000630445094236531, 'male': 0.0002960331557134399}

The product of conditional probabilities is a very small number. So, we should  calculate sum of logarithms (use the log space) to avoid underflow error. Logarithm of a number < 2 is a negative number. 

Also: $a, b < 2$ and $a < b$, => $log(a) < log(b)$. 

Thus, the gender with greater log probability will be the answer.

In [44]:
def nb_predict(model, document):
    """ predict gender for document
    """
    probabilities = defaultdict(lambda : 1)
    words = word_search_re.findall(document)
    for word in set(words):
        probabilities['male'] += np.log(model[word].get('male', 1e-15))
        probabilities['female'] += np.log(model[word].get('female', 1e-15))
    likely_genders = sorted(probabilities.items(), key=itemgetter(1), reverse=True)
    return likely_genders[0][0]

Copy post from the dataset for *male* to try prediction.

In [45]:
post = "'\\n\\n\\nSlashdot raises lots of  urlLink interesting thoughts about banner ads .  The idea is to let users control the ad delivery, and even to allow users to comment on ads.\\n\\n\\n\\n\\n\\n\\nurlLink  The Merchants of Cool  , a Frontline documentary featuring Mindjack advisory board member Douglas Rushkoff, is on PBS tonight.  Check your local listings for the time.\\n\\n\\n\\n\\n\\n\\nurlLink ATMs dispensing music?   I don\\'t quite see the logic in that.  I\\'m not entirely against paying a nominal fee for music, or any other media, but if I do have to pay for it I\\'d be much more likely to buy stuff from my own PC.\\n\\n\\n\\n\\n\\n\\nMy chair started squeaking a few days ago and it\\'s driving me nuts!\\n\\n\\n\\n\\n\\n\\nThe New York Press has a  urlLink lenghty article  about the post dotcom crash scene that\\'s worth reading.\\n\\n\\n\\n\\n\\n\\nFrancis Collins\\'  urlLink 10 Most Surprising Things About the Human Genome .\\n\\n\\n\\n\\n\\n\\nurlLink FILMSPEED  has great movies you can download for free.  The Third Man, in particular, is one of my favorite films of all time.\\n\\n\\n\\n\\n\\n\\nFor sale:  An STB nVidia ZX 8MB 3D Accelerator (AGP) and a 40X NEC CD-ROM drive.  Interesting trades considered.\\n\\n\\n\\n\\n\\n\\nI\\'ve started uploading stuff to my live365.com radio station.  There\\'s not much there yet but you\\'re welcome to  listen .  (cable/dsl or higher required)\\n\\n\\n\\n\\n\\n\\nFEED\\'s latest special issue is an excellent  urlLink examination of video games , with contributions from Carl Steadman, Steven Johnson, Justin Hall and others.  Don\\'t miss it.\\n\\n\\n\\n\\n\\n\\nNew DVD!   urlLink The Maltese Falcon .\\n\\n\\n\\n\\n\\n\\nI once had a personal homepage, long before weblogs came into fashion, but never managed to update more than a couple of times a year.  Here\\'s hoping I\\'ll be a bit more successful with this.  Big thanks to  urlLink Bopjet  for hosting this and  urlLink Mindjack .\\n\\n\\n\\n\\n\\n\\nurlLink The Register reports on \"war driving\"  - the wireless equivalent of war dialing.  Instead of having your modem dial into thousands of networks until you get in, you just drive within range of a wireless net with your wireless-equipped laptop and hack away.    related:   urlLink The latest issue of CIO  has a great feature on wireless.\\n\\n\\n\\n\\n\\n\\nurlLink Short documentary on digital filmmaking and Star Wars Episode II.\\n\\n\\n\\n\\n\\n\\nurlLink Rumour has it  that the parent companies of  urlLink Inside.com  and  urlLink Brill\\'s Content  will be announcing a merger as early as next Monday.  Apparently the two print magazines would fold into one called  Brill\\'s Inside Content  and the two websites would form  Media Central Inside.com .  Rolls right off the tongue, doesn\\'t it?\\n\\n\\n\\n\\n\\n\\nurlLink Destroy All Monsters - Survivor: Monster Island .  My money\\'s on Mothra.\\n\\n\\n\\n\\n\\n\\nI just spent some time playing with the beta version of  urlLink Atmosphere , Adobe\\'s new 3D environment software, and thought I\\'d give a couple of quick impressions.    I\\'ve always been interested in VR and spent quite a bit of time working in VRML, but I think the inherant problem with VRML was that not enough attention had been given to community.  You could build all these really cool places but people couldn\\'t interact in them.  You also couldn\\'t easily move from one 3D environment to another without going back and forth between 2D web pages.  I was involved with a project that tried to solve some of these problems with VRML (anybody remember Terra Vista?) but it didn\\'t as far as we would have liked.  Atmosphere appears to have addressed these problems.  It\\'s not the first product to do so, but maybe Adobe has enough muscle for it to gain wide acceptance.\\n\\n\\n\\n\\n\\n\\nThanks to  urlLink Onebox.com  I should now be able to receive faxes.  Not suprisingly, they don\\'t have local phone numbers for Fredericton so I chose a New York City area code.  If anyone wants to give it a whirl, my number is: (212) 894-3748 x2690.\\n\\n\\n\\n\\n\\n\\nWell, I was supposed to be working.  But, through a long trail of procrastination, I ended up redesigning this site.\\n\\n\\n\\n\\n\\n\\nQuotable Mindjack!  From Mike Sugarbaker\\'s  urlLink review of Lemon :  \"If you have no patience for the lengthy ruminations of brilliant madmen, Lemon isn\\'t for you. But you read Mindjack, so you\\'re probably into that sort of thing, right?\"\\n\\n\\n\\n\\n\\n\\nMindjack is featured in the links section of this month\\'s  urlLink Netlife magazine .  It doesn\\'t seem to be online yet so here\\'s a scan:\\n\\n\\n\\n\\n\\n\\nWell, the big Yahoo news yesterday was that Tim Koogle is stepping down as CEO and that they\\'ll miss forecasts in the first quarter.   urlLink Today the stock tumbled .\\n\\n\\n\\n\\n\\n\\nurlLink The Times reports  that the  urlLink Trogan Room Coffee Machine  webcam is going offline for good later this year.  That pot should go in a museum.\\n\\n\\n\\n\\n\\n\\nurlLink It looks like  there\\'s some big news due from Yahoo later today.\\n\\n\\n\\n\\n\\n\\nThe new issue of  urlLink Mindjack Magazine  is now online, featuring a first for us - a live concert review.  With original photos even.\\n\\n\\n\\n\\n\\n\\nIt\\'s not good when a box of Sinutab describes your day.   Sinus pain, headache, congestion, itchy, watery eyes.   Ugh.\\n\\n\\n\\n\\n\\n\\nurlLink The End of Free  - chronicling the demise of free stuff on the net.\\n\\n\\n\\n\\n\\n\\nI\\'m not sure I like the direction  urlLink FEED just turned .  \"The filter\", a new weblog linking to outside content is now featured more prominently than FEED\\'s original content.  It\\'s also not clear what the difference is between the filter and  urlLink Plastic .\\n\\n\\n\\n\\n\\n\\nHey now.   urlLink Jon Lebkowsky  has started a blog.  Coolness and wisdom is to be found within.\\n\\n\\n\\n\\n\\n\\nWhat looks like the  urlLink new PalmOS PDA  from TRG looks mighty impressive.  The official launch is rumored to be on Monday.  If it\\'s priced reasonably enough it might get me to upgrade from my 512k PalmPilot Personal.\\n\\n\\n\\n\\n\\n\\nurlLink Stick figure fighting action  like you\\'ve never seen before.\\n\\n\\n\\n\\n\\n\\nEvan Williams of Blogger has more on  urlLink How the Trellix Deal Happened -- and What It Means .\\n\\n\\n\\n\\n\\n\\nNeed a  urlLink logo ?  Maybe a  urlLink website ?   urlLink Email me .  I will work for  urlLink toys .\\n\\n\\n\\n\\n\\n\\nIt was just announced that  urlLink Trellix is licensing Pyra\\'s Blogger technology , giving Pyra a much needed cash infusion.  Dan Bricklin has a fascinating account of  urlLink how the deal happened  on his site.\\n\\n\\n\\n\\n\\n\\nI recently got the  12 Monkeys DVD , which is excellent.  It\\'s worth picking up just for the included feature-length documentary,  The Hamster Factor and Other Tales of Twelve Monkeys , about the making of the film.\\n\\n\\n\\n\\n\\n\\nRedesigned again.\\n\\n\\n\\n\\n\\n\\nurlLink Inside.com reports  that Economist business editor Chris Anderson has been named editor of Wired, replacing Katrina Heron.\\n\\n\\n\\n\\n\\n\\nThe new issue of  urlLink Digital Web Magazine  is out today.  Lots of interesting stuff.\\n\\n\\n\\n\\n\\n\\nurlLink This is  one of the reasons I\\'m a vegetarian. (washingtonpost.com)\\n\\n\\n\\n\\n\\n\\nJustin\\'s  urlLink back from Japan  with stories and pictures.\\n\\n\\n\\n\\n\\n\\nSomething screwy\\'s going on with my computer.  Things are crashing, my mouse is stuttering, MP3s are skipping when they didn\\'t before, and sometimes it just slows to a crawl.  I\\'ve got a nagging feeling that I\\'m going to have to do a complete format and reinstall Windows but I  really  don\\'t want to do that.\\n\\n\\n\\n\\n\\n\\nurlLink Mikel.org  pointed me to this rumor that  urlLink Automatic Media is in trouble .  Automatic is the company that consists of Feed, Suck and Plastic.  Hopefully it is just a rumour.\\n\\n\\n\\n\\n\\n\\nurlLink Home of the Underdogs  is an amazing resource for anyone that loves computer games.  Not only do they have hundreds of games you can download which are no longer available from the companies, they also have a wealth of information about many of the games.  It\\'s not entirely legal, but I think it\\'s a great public service.  It\\'s even better now that they have one of my favorite games,  urlLink Gabriel Knight .\\n\\n\\n\\n\\n\\n\\nurlLink There\\'s a great article  by William Gibson in the The Observer\\'s  urlLink Special Japan Issue .\\n\\n\\n\\n\\n\\n\\nurlLink \\'We reinvented culture the way we wanted it - with great big shoes\\' .  David Bowie writes about glam in today\\'s Guardian.\\n\\n\\n\\n\\n\\n\\nIt\\'s been a while since I played around with customizing  urlLink My Yahoo!  so I don\\'t know how new this is, but you can now use themes in additoin to plain colors.\\n\\n\\n\\n\\n\\n\\nurlLink Streaming video from the Reboot conference , featuring Justin Hall, Dave Winer, Evan Williams, Douglas Rushkoff and others.  It\\'ll make ya think.\\n\\n\\n\\n\\n\\n\\nMindjack is now in deep development mode and, through the magic of blogs, you can  urlLink follow the action as it happens .\\n\\n\\n\\n\\n\\n\\nurlLink Shift is back  with a design that looks  urlLink vaguely familar .  It looks good but those javascript scroll boxes are slooooow.\\n\\n\\n\\n\\n\\n\\nIf I had heard Doves  urlLink Lost Souls  album last year, I would have included on my  urlLink best of 2000 list .    Hawksley Workman\\'s  (Last Night We Were) The Delicious Wolves  will probably be on this year\\'s list.\\n\\n\\n\\n\\n\\n\\nCNET has  urlLink video of Steve Jobs tour  of Apple\\'s new retail stores.\\n\\n\\n\\n\\n\\n\\nurlLink Ain\\'t It Cool News  has some great clips and pictures from the new Coen Brothers film, The Man Who Was Not There.  I\\'m excited.\\n\\n\\n\\n\\n\\n\\nYet another redesign.  I think I\\'ll stick with this one for a while.\\n\\n\\n\\n\\n\\n\\nSure, it\\'s advertising.  But it\\'s  urlLink damn good advertising.\\n\\n\\n\\n\\n\\n\\nDSL!\\n\\n\\n\\n\\n\\n\\nFeed has a great  urlLink interview with Cory Doctorow  today.     The overlap between Doctorow\\'s day job at OpenCola and his night job writing fiction makes for a unique combination: It\\'s almost like William Gibson launching a company that sells neural plugs and stim-sims.\\n\\n\\n\\n\\n\\n\\nDouglas Rushkoff\\'s Frontline documentary,   urlLink The Merchants of Cool   is being rebroadcast tonight on PBS (check your local listings).  Set your Tivo or VCR, or watch it as it\\'s on even.\\n\\n\\n\\n\\n\\n\\nI\\'m DSLless for 10 days.  I suppose compared to most other companies that\\'s pretty snappy service, but it\\'s still 10 days without DSL.\\n\\n\\n\\n\\n\\n\\nurlLink FEED  gave the official word today that they\\'re sutting down, at least for the foreseeable future.  The same goes for  urlLink Suck , although it looks like  urlLink Plastic  will be continuing for now.    Both FEED and Suck have been online for six years, and have been excellent throughout.  Hopefully they can pull out this funk and come back in one form or another in a few months.\\n\\n\\n\\n\\n\\n\\nJason Kottke has some  urlLink great pictures from the Louvre  today.\\n\\n\\n\\n\\n\\n\\nurlLink Great roadtrip article by Mark Frauenfelder  in One magazine.   \"A half-century ago, the streets of Southern California were filled with flamboyant towers of consumerism -- spires and obelisks and beacons -- transmitting neon-fortified messages of optimism to families riding in cars with space-age tailfins. Life was good, and the marvels of modern science were making a world as new as tomorrow.\"\\n\\n\\n\\n\\n\\n\\nBill Frisell\\'s latest album  Blues Dream  arrived in the mail today and it only confirmed what I suspected.  I have got to get more Frisell.\\n\\n\\n\\n\\n\\n\\nurlLink blogdex  is a spider that crawls blogs and generates a list of the most linked to sites.  Ten bucks says blogdex itself is number one by tomorrow.\\n\\n\\n\\n\\n\\n\\nliving dangerously  I haven\\'t hotsynced my PalmPilot in a couple of months.  How long can it go before craping out?  Stay tuned.\\n\\n\\n\\n\\n\\n\\nThis year\\'s Webzine conference in NYC looks like it went over well.  We have coverage in the  urlLink current issue of Mindjack .\\n\\n\\n\\n\\n\\n\\nLooking for some summer reading?   urlLink Canadian Media Theorists: Canadian Media Resources .     found via  urlLink mikel.org\\n\\n\\n\\n\\n\\n\\nI\\'ve been reading  urlLink Boing Boing  since before it was a weblog but I just noticed the links to  urlLink Mark  and  urlLink Cory\\'s  garage and flea market finds today.\\n\\n\\n\\n\\n\\n\\nMan.  I uninstalled McAfee Viruscan and my computer seems twice as fast.  Now, to find a virus scanner with less of a memory appetite.\\n\\n\\n\\n\\n\\n\\nurlLink Mindjack is back  and there\\'s more to come.\\n\\n\\n\\n\\n\\n\\nThey don\\'t make keyboards like this anymore.  This is the Underwood Portable I mentioned the other day.\\n\\n\\n\\n\\n\\n\\nurlLink StarOffice 6  looks like it will be a big improvement over the current version.  I\\'ve been using StarOffice as an alternative to PowerPoint, but the sluggishness mentioned in this article has stopped me from using it regularly for much else.\\n\\n\\n\\n\\n\\n\\nurlLink Synapse  is now offiicially open.  Come and stop by.\\n\\n\\n\\n\\n\\n\\nSynapse is live!  Almost.  It will publicly launch late tonight, but we\\'re looking to get some conversations going before that.  If you\\'d like to help out  go to  urlLink synapse.mindjack.com  and join now.\\n\\n\\n\\n\\n\\n\\nMy copy of  urlLink Words! Words! Words!  arrived the other day.  It is good.\\n\\n\\n\\n\\n\\n\\nI seem to have unintentionally started a typewriter collection.  I was given an Underwood Portable Typewriter in near perfect condition yesterday.\\n\\n\\n\\n\\n\\n\\nMore cool stuff.  I picked up this Egyptian painting on papyrus at an estate auction the other day.        It\\'s actually in better condition than this picture makes it look.   urlLink Here\\'s an image  I found of it online.\\n\\n\\n\\n\\n\\n\\nWe\\'ve finally set a definite launch date for Synapse, Mindjack\\'s long in development online community.  Monday, next week.    Stay tuned.  It\\'ll be good.\\n\\n\\n\\n\\n\\n\\nurlLink APOD: 2001 August 20 - The Lagoon Nebula in Three Colors .  My new wallpaper.\\n\\n\\n\\n\\n\\n\\nI went to Atlantic Canada\\'s largest flea market in Sussex, New Brunswick yesterday and got a sun burn and this:                  For 25 bucks.  According to  urlLink this site  it\\'s a 1911 model.  It works great apart from a few sticking keys.  Now to find some ribbon for it.\\n\\n\\n\\n\\n\\n\\nStaples was selling  urlLink Kensington Videocams  for $20cdn yesterday.  The result?  This ain\\'t a webcam.   Reload if you want to, but it ain\\'t changin\\'.\\n\\n\\n\\n\\n\\n\\nHoly shit.  According to Advertising Age,  urlLink the Industry Standard is ceasing publication .\\n\\n\\n\\n\\n\\n\\nurlLink blogBuddy  looks useful.  It\\'s a Windows-based Blogger interface.-  urlLink via Evhead\\n\\n\\n\\n\\n\\n\\nVia  urlLink Doc Searls Weblog  I discovered a  urlLink great weblog all about 802.11b .\\n\\n\\n\\n\\n\\n\\nurlLink The Village Voice: Features: High Speed, Freed by Peter Meyers  -  Good article on 802.11b wireless networks in NYC.\\n\\n\\n\\n\\n\\n\\nNew!  In the mini-blog over there you can take a peek at some of my recent media consumption.  Even better, if you buy it too I\\'ll get $$.  Which may be some added incentive for me to keep this thing updated.\\n\\n\\n\\n\\n\\n\\nurlLink Today\\'s DaveNet  is a great explanation of  urlLink Blogger API  and what it means.\\n\\n\\n\\n\\n\\n\\nMy copy of  urlLink Words! Words! Words! Magazine  is on it\\'s way to me from New Zealand.  Is yours?\\n\\n\\n\\n\\n\\n\\ntoday\\'s best spam    DEAR SIR / MADAM :  HOW DO YOU DO !  WE CAN SUPPLY HIGH QUALITY FLYING SHOES, SUCH FLYING SHOES LET YOU BOTH CAN WALK NORMALLY, AND CAN SLIDE AS FLYING. IT IS A SORT OF FASHIONABLE PRODUCT.  THE FOB PRICE IS USD 7.70 PER PAIR, WELCOME ORDER !  PLEASE LOOK THE DETAILS WITH PHOTOS FROM : http://www.chunpai.com/flyingshoe.htm   ALSO WE HAVE WOOD SHOES, BAMOO MATS, WOOD FLOORING, ALUMINU PLASTIC DECORATE PANELS, ETC, LOOK THEM FROM : http://www.chunpai.com/first-english.htm  BEST REGARDS !  MR.TAN LONG  ( SATRAP ) GUILIN LINGUI SHENGPING XIAOXUE BAMBOO & WOOD MANUFACTORY ADDRESS: No.229 RONGSHAN ROAD,LINGUI,GUILIN,GUANG,CHINA TEL / FAX: +86-773-5592687 E-MAIL: chunpai@public.glptt.gx.cn WEB SITE: http://www.chunpai.com/first-english.htm\\n\\n\\n\\n\\n\\n\\nNow in Mindjack is  urlLink an excerpt  from Cory Doctorow\\'s novel-in-progress, Eastern Standard Tribe.  Cory\\'s also taking part in a discussion in  urlLink Synapse .\\n\\n\\n\\n\\n\\n\\nYou skip reading a few issues of Wired and you miss something really great:   urlLink My Own Private Tokyo  by William Gibson\\n\\n\\n\\n\\n\\n\\nurlLink Coltrane at 75: the Man and the Myths  - New York Times\\n\\n\\n\\n\\n\\n\\nurlLink Photgraphica\\'s been down for a while , but in it\\'s absence I\\'ve discovered two other great photologs (when they started being called that I\\'m not sure).   urlLink PixelPile  and  urlLink Common Threads .  Now, if only I had a digital camera.\\n\\n\\n\\n\\n\\n\\nIt Came From British Columbia!   urlLink Great space art  made with found objects. -  via  urlLink robots.net\\n\\n\\n\\n\\n\\n\\nMindjack\\'s been  urlLink slashdotted .  And  urlLink BoingBoinged .\\n\\n\\n\\n\\n\\n\\nurlLink prolific  has David Bowie\\'s excellent  urlLink cover of America  by Paul Simon in real audio.\\n\\n\\n\\n\\n\\n\\nNovemember is  urlLink National Novel Writing Month .  The goal is to write a 200 page, 50,000 word novel by November 30th.  I think I might start by trying to write a short story in a month.\\n\\n\\n\\n\\n\\n\\nurlLink A great gallery of low-res pictures  taken with a PenCam.  Not to mention a very cool interface.    urlLink via boingboing\\n\\n\\n\\n\\n\\n\\nThanks to the very cool  urlLink BlogBack , comments are back.\\n\\n\\n\\n\\n\\n\\nurlLink Odyssey made it .  Yes, I am a space geek.\\n\\n\\n\\n\\n\\n\\nurlLink Watch as Odyssey enters Mars\\' orbit .  Cross your fingers.\\n\\n\\n\\n\\n\\n\\nurlLink Mindjack  is looking for writers, preferably ones that can commit to writing at least one article per month.  We can\\'t pay anything right now, but I can promise that we\\'ll pay you as soon as we make any money ourselves.  If you\\'re interested, drop me an email:  urlLink editor@mindjack.com .  Include a few samples of your writing if you can, and I\\'ll get back to you real soon.  Feel free to pass this along to anyone you think might be interested.\\n\\n\\n\\n\\n\\n\\nSpecial features on the new  urlLink Monty Python and the Holy Grail DVD :  \\x95 Region 1 encoding (US and Canada only) \\x95 Color, Closed-captioned, Widescreen \\x95 Commentary by Terry Gilliam & Terry Jones, plus general complaints and back-biting by John Cleese, Eric Idle & Michael Palin \\x95 Theatrical trailer(s) \\x95 High definition widescreen presentation with pink frilly edges \\x95 Subtitles for People Who Don\\'t Like the Film (taken from Shakespeare\\'s Henry IV, part II) On-Screen Screenplay: read the screenplay while you watch the film \\x95 Exciting \"Follow The Killer Rabbit\" Feature! \\x95 A special version for the Hard of Hearing \\x95 A glorious extra 24 seconds absolutely free \\x95 Three Mindless Sing-Alongs \\x95 Join Michael Palin and Terry Jones in their special documentary: The Quest for the Holy Grail Locations \\x95 How To Use Your Coconuts (an educational film) \\x95 Monty Python and the Holy Grail In Lego \\x95 \"On Location with The Pythons\". Genuine 18 min. location report made in 1974 by BBC Film Night. (broadcast December 19, 1974) \\x95 An Interactive Cast Directory - discover just how many roles Michael Palin plays \\x95 Tons of Terry Gilliam\\'s original sketches plus posters \\x95 Behind-the-Scenes Photos - hitherto unseen by human eyes \\x95 A Load of Old Rubbish - a surprise package of mystery items specially included for the mentally challenged \\x95 How the directors\\' recce used up the budget \\x95 Widescreen letterbox format  I can\\'t wait.\\n\\n\\n\\n\\n\\n\\nCory Doctorow has started recording  urlLink audio versions of all his stories , starting with   To Market, to Market, The Branding of Billy Bailey , which I\\'m downloading now.  Where do you find the time, man?\\n\\n\\n\\n\\n\\n\\nMatt Hinrichs (a.k.a  urlLink scrubbles ) has done some great illustrations for  urlLink Synapse .  Join now to see them all.   urlLink\\n\\n\\n\\n\\n\\n\\nRedesigned.  Again.\\n\\n\\n\\n\\n\\n\\nCrap.  I ordered the new Citizen Kane DVD  urlLink from one place  the other day, and today found it ten bucks cheaper  urlLink somewhere else .\\n\\n\\n\\n\\n\\n\\nurlLink The Infinite Matrix  is back.  Whoohoo!\\n\\n\\n\\n\\n\\n\\nNow in Mindjack:  urlLink Marketing The X , an examination of Microsoft\\'s marketing of the Xbox and Windows XP, and the first of hopefully many articles by Jane  urlLink umami tsunami  Pinckard.\\n\\n\\n\\n\\n\\n\\nurlLink Justin\\'s site  is turning into a wonderful resource on all things cool and Japanese.\\n\\n\\n\\n\\n\\n\\nI have got to start writing more.  Anybody want to pay me to do it?\\n\\n\\n\\n\\n\\n\\nI watched  urlLink No Maps For These Territories  the other night, a great documentary that director Mark Neale was kind enough to send me (running a magazine has it\\'s perks).  The whole thing consists of William Gibson sitting in the back of a limo musing on everything from cyberspace to religion.  I\\'ll be doing a full review in Mindjack within the next couple of weeks.\\n\\n\\n\\n\\n\\n\\nHere\\'s what the Slashdot effect looks like.\\n\\n\\n\\n\\n\\n\\nNew Mindjack today.     urlLink Twenty Twenty : Astronomical Vision by Chris McKinstry   \"Very early on a cold February morning in 1979 I looked with one of my highly astigmatic eyes through my first professional telescope at a setting Jupiter. Even with my flawed vision, the long white metal tube so finely manufactured it could have been a piece of sophisticated medical equipment, let the cloud bands of our solar system\\x92s largest planet and the bright points of a handful of its moons into my mind, making me feel a shadow of the shock that Galileo must have felt when he saw the same sight before any of us. \"\\n\\n\\n\\n\\n\\n\\nI redesigned  urlLink Mindjack  a bit.  It should load quite a bit faster now, and look better at lower resolutions.    New this week,  urlLink Design for Community  reviewed by David Brake.  Next week, an interview with Derek Powazek.\\n\\n\\n\\n\\n\\n\\nThe new Infinite Matrix is up with a long excerpt of Cory Doctorow\\'s new novel,  urlLink Down And Out In The Magic Kingdom .  The Liberty Square ad-hocs were the staunchest conservatives in the Magic Kingdom, preserving the wheezing technology in the face of a Park that changed almost daily. The newcomer/old-timers were on-side with the rest of the Park, had their support, and looked like they might make a successful go of it.\\n\\n\\n\\n\\n\\n\\nI\\'m the top left corner on Cory Doctorow\\'s  urlLink Four blogs on one page , which is one excuse why I haven\\'t been posting to this blog very much lately.  I\\'ll try to give myself a kick in the pants over the x-mas break and get this thing going again.\\n\\n\\n\\n\\n\\n\\nurlLink Wonderful gallery  of Atari concept illustrations.   via  urlLink BoingBoing\\n\\n\\n\\n\\n\\n\\nI redesigned the Mindjack front page a bit to include the entire  urlLink Daily Relay blog , rather than just excerpts.\\n\\n\\n\\n\\n\\n\\nurlLink denizen  is a new weblog about classic gaming from the creator of  urlLink Home of the Underdogs .\\n\\n\\n\\n\\n\\n\\nNow in Mindjack, a review of the  urlLink Xircom Wireless LAN Module  for Palm m500 handhelds.  A subject that only Cory Doctorow could make this interesting.   Four or five years ago, I had a sea-change in my computing experience. One day, I realized that I had started to view any computer not connected to the Internet as just a fancy game-console-cum-jumped-up-typewriter. Working on an airplane or from a caf\u00e9 just felt wrong.    This feeling deepened and magnified a thousandfold the day I got my first iBook and Airport Base Station. If you\\'ve used wireless networking on a laptop, you know what I\\'m talking about -- snatching the Internet out of the very air is an experience you won\\'t go back from willingly (another one is TiVo -- I\\'ve stopped watching TV in hotel rooms because I can\\'t choose among thirty hours of custom-recorded programming and skip commercials). Just as TiVo ruined me for dumb TV, 802.11 ruined me for wired networking or no networking at all.    And it keeps on getting better. 802.11 isn\\'t just a technology, it\\'s a movement, an ad-hoc world of open base-stations around the world. Just haul out your 802.11-equipped device and start hunting about for a network. If you\\'re in a major city, chances are you\\'ll find one before you go a block. Forget 3G and Blackberry and all those other pale imitations of connectivity: community wireless is the real shit: fast, unmetered, insecure and out of control.\\n\\n\\n\\n\\n\\n\\nMindjack T-shirts are  urlLink now available .  These are high-quality, genuine screenprinted shirts, not the Cafe Press stuff.  Thanks  urlLink Brunetto !  If you\\'re looking to get some shirts printed I highly recommend them.     Also available are  urlLink Mindjack coffee mugs , featuring an illustration by  urlLink Matt Hinrichs .  These are Cafe Press ones.  Please buy some.\\n\\n\\n\\n\\n\\n\\nBuild your own robot!  urlLink CAM-08 \\x96 The Perpetual Humanoid Model .       I want one.  Especially if it comes with the sound effects in this  urlLink video .\\n\\n\\n\\n\\n\\n\\nFebruary needs at least five more days.  I still have loads of work that was supposed to be done by now.\\n\\n\\n\\n\\n\\n\\nWell, this blog is back and oranger than ever.  Now just to get  urlLink Daily Relay  back up and running.\\n\\n\\n\\n\\n\\n\\nMy mindjack.com email should be working now.  Let me know if you still have problems.\\n\\n\\n\\n\\n\\n\\nCrap.  All my @mindjack.com email addresses are messed up.  If you need to reach me please use donald@melanson.ca.\\n\\n\\n\\n\\n\\n\\nWho\\'d have thought Mattel is still making classic  urlLink electronic football games .        I used to think these were the most amazing things ever.  Then I got an Atari 2600.\\n\\n\\n\\n\\n\\n\\nThere\\'s a great little Chinese grocery down the street from my apartment where I always find something I\\'ve never tried before.  Today:  Iron Buddha Tea.     It\\'s really good tea.  Try it.\\n\\n\\n\\n\\n\\n\\nurlLink Jim Lai reports on Digifest 2002  in the new issue of Mindjack.  \\t  I didn\\'t have much in the way of expectations going in to Digifest 2002; pretty pictures, glitzy art/advertising pieces, and possibly some cool tech. I was pleasantly surprised by the breadth and depth of the presentations.   Wednesday night.  Hit the media session and got to see some graphics on a really big screen. The theme of this international digital media festival was the \"4-D challenge\", to go beyond the third dimension. I would deem this festival to be a success overall, as by the end of the festival it would be apparent that there was much speculation on what constitutes this fourth dimension. Now that I\\'ve spoiled the ending and gotten that out of the way, on with the festival itself.  The original vision was for local (Toronto) participation, but it quickly expanded in scope with the arrival of international submissions. There was a soir\u00e9e before the keynote where attendees could snack on catered food and mingle. Quite a few people were there, many of them either dressed hip or stylishly. I almost felt that I was underdressed, but I just put on my best casual face and it was all cool. Business cards flowed like water.\\n\\n\\n\\n\\n\\n\\nI got an iBook!  OSX sure takes some ajusting to but I\\'m really impressed with it so far.\\n\\n\\n\\n\\n\\n\\nurlLink Neil Gaiman  on whether someone should or should not be a writer:  And in answer to your question, if you worry about job security as a writer, that\\'s a very good indication that you aren\\'t a writer. Hating writing is another good one, although I know writers who hate writing. (They are miserable.)\\n\\n\\n\\n\\n\\n\\nIn the new Mindjack: a  urlLink report on SXSW Interactive  by Jon Lebkowksy, with photos by  urlLink Paul Bausch .   This year\\'s South by Southwest Interactive conference was lean and mean - attended mainly by the core group of edgy \\'net whackadistas, the conference had an interesting vibe, like \"Wow, glad the goddam dotcom splurge is over, let\\'s get back to what we were doin\\'\\x85\" And what we were doin\\' had real depth, it was way more compelling than ecommerce or net.publishing, the kinds of projects MBAs brought to the table when they started calling the Internet an \\'industry\\' and creating the concept of the IPO casino.\\n\\n\\n\\n\\n\\n\\nMindjack is back from a short break with a great article by Douglas Rushkoff:  urlLink Conforming to the Machine .\\n\\n\\n\\n\\n\\n\\nIf you aren\\'t reading  urlLink futurefeedforward , you should be.   NEW YORK--ABC news magazine \"20/20\" reporter John Stossel, accidentally decapitated late last month while shooting a segment \"debunking the myth of wind power,\" returns to the air Wednesday in a special interview with 20/20 anchor Barbara Walters. \"It\\'s really an amazing story,\" explains Walters. \"Most people wouldn\\'t even survive decapitation, let alone have the guts, the determination, to keep doing their jobs. It\\'s a real triumph of the human spirit.\"\\n\\n\\n\\n\\n\\n\\nEveryone\\'s favorite Cory Doctorow, Cory Doctorow, has gotten a  urlLink new gig  at the EFF, which reminds me that I haven\\'t suported them nearly enough.  So, for the rest of March I\\'ve decided to donate $5 from each  urlLink Mindjack t-shirt  sold to  urlLink EFF .\\n\\n\\n\\n\\n\\n\\nurlLink Daily Relay  is finally back up.  We\\'re using Blogger as an interim solution and will probably switch to Blogger Pro if I can\\'t get the Moveable Type problems sorted out.\\n\\n\\n\\n\\n\\n\\nThis week\\'s Mindjack:  urlLink Reading McLuhan  by Melanie McBride.  If there\\'s a message of the \\'for dummies\\' age it\\'s that nothing is beyond our grasp. And our desire to believe this is reinforced by trends like usability, which privilege economy over elucidation. No one anticipated it all better than Marshall McLuhan, who whittled big insights into sound bites in order to engage an audience beyond the lecture halls of the University of Toronto. With the help of Tom Wolfe and others, the scholarly McLuhan became a cool media prophet. It was, and still is, a practical strategy in anti-intellectual times. But in the process, much of McLuhan\\'s meaning has been reduced to a one-liner. This has as much to do with the absence of commentary on McLuhan\\'s literary, philosophical and cultural influences as it does with the way his work is taught.\\n\\n\\n\\n\\n\\n\\nThe new issue of Mindjack deals with one of my favorite topics:  urlLink Robots!\\n\\n\\n\\n\\n\\n\\nMy submission to the  urlLink Burn Baby Burn  mix CD swap, with dispraportionate amounts of Bowie and Reed, of course.   urlLink     (click to read tracklist)\\n\\n\\n\\n\\n\\n\\nMindjack advisory board member Douglas Rushkoff has started  urlLink a blog .   Expect good things.\\n\\n\\n\\n\\n\\n\\nNot that you\\'\\'d expect anything less, but the  urlLink Criterion Collection  website is really excellent.  It\\'s similar the old  urlLink Criterion Goes To The Movies  CD-ROM, which is also worth checking out.  Although it takes some work to get it to run on any computer made in the last six or seven years.  Both are filled with great essays on some of the best moves ever.\\n\\n\\n\\n\\n\\n\\nWe\\'ve got a  urlLink great interview with Simon Singh , author of  The Code Book , in the new issue of Mindjack.\\n\\n\\n\\n\\n\\n\\nGadzooks, it\\'s been a long time since I last posted.  Sorry.  I interview  urlLink Derek Powazek  in the new Mindjack, in which he talks about online community, his book and weblogs as journalism.\\n\\n\\n\\n\\n\\n\\nMindjack is trying out a new guerrila advertising campaign.  Feel free to use either of this ads on your weblog or website.         More to come soon.\\n\\n\\n\\n\\n\\n\\nurlLink Mindjack  is back!   Now to get back to work on this site.\\n\\n\\n\\n\\n\\n\\nurlLink Mindjack interviews Warren Ellis!     So why aren\\'t you reading comics these days?  Really, it doesn\\'t seem to me that there are many comics being written for me. I want something with a little more muscle and bite than standard-issue power fantasies, whimsical romance, the autobiographies of people who never do anything and things with elves. The Western medium has cycles, and it\\'s currently in a creative downturn. That doesn\\'t mean there isn\\'t excellent work being done. That simply means there\\'s not much of it. And, on a personal level, little of it is talking to me. Not many comics reflect the fact that I live in a multicultural society fitted with a global communications net, nor do they reflect the fact that I don\\'t own a pair of Superman underpants.\\n\\n\\n\\n\\n\\n\\nHoward Rheingold  urlLink interviewed on NPR  (RealAudio link).  I still haven\\'t gotten my copy of  urlLink Smart Mobs  yet, but hope to soon.\\n\\n\\n\\n\\n\\n\\nMindjack\\'s Jim Lai checks out an art exhibit in the form of a billboard:  Transmedia 2002\\'s  urlLink 15 Seconds of Fame.\\n\\n\\n\\n\\n\\n\\nNow in Mindjack:   urlLink Cybersecurity Nosebleed  by Sarah Granger \"Security is the newest over-hyped commodity in the United States. I keep visualizing driving down highway 101 past billboards that read: \"Big government wants big security\" along with a picture of an ape beating on its chest. Somehow as seriously as they say they are, I\\'m not convinced.\"\\n\\n\\n\\n\\n\\n\\nurlLink Mindjack\\'s back , with a proper relaunch this time after a bit of a slow start last month.  We\\'ve got a lengthy article on Lawrence Lessig by Bryan Alexander this issue, plus my review of the neat little l\\'espion digital camera.  Spread the word!\\n\\n\\n\\n\\n\\n\\nI\\'m kicking myself for missing the  urlLink David Bowie concert  on BBC Radio 2 today.  But at least I was able to find a couple of the songs  urlLink somewhere else .   Bewlay Brothers is amazing.\\n\\n\\n\\n\\n\\n\\nIn case anyone\\'s sitting on the edge of their site, rest assured the Mindjack relaunch is still on track for Monday.  Get the word out.\\n\\n\\n\\n\\n\\n\\nI can\\'t get enough of CBC\\'s  urlLink 50th anniversary  specials.  I wish they\\'d start a cable channel and show nothing but stuff from the 50s and 60s (and 70s and 80s too).\\n\\n\\n\\n\\n\\n\\nI just got  urlLink Syberia  and  urlLink Post Mortem , two promising new adventure games from Montreal-based developer  urlLink Microids .  As you might be able to judge from some of my posts here, I\\'m a big fan of adventure games.  The traditional kind, where you can carry an enormous inventory in your pants, and explore the game at your own pace.  I\\'ve been thinking a lot about the genre lately and plan to write something about it soon, aside from reviews of the aforementioned games.  I\\'m interested in hearing your thoughts on adventure games.  Do you think there\\'s a place for them today?  What are your favorite games?    src=\"http://www.quicktopic.com/18/iaMjuWreTAS.js\">   urlLink Comments   document.write(\"(\" + ct_18_iaMj + \")\");\\n\\n\\n\\n\\n\\n\\nThis doesn\\'t bode well for me getting much work done this weekend.  The Adrenaline Vault\\'s  urlLink Best Shareware of 2002\\n\\n\\n\\n\\n\\n\\nJustin Hall on South Korea:  urlLink korea will be televised .  TVs in cabs, TVs in subways, and stacks and stacks of tiny, tiny televisions to take with you.   But I was still able to enjoy more television, this time in the Seoul subway. Televisions up in the subways, instead of static ad bulletin boards! This is the future! It\\'s fun to see and fun to watch now. Soon it might be annoying or normal. But this first brush with more constant televisual stimulation was definitely exciting.\\n\\n\\n\\n\\n\\n\\nI\\'d like to comment on  urlLink Movielink , the new studio-approved online movie rental service, but unfortunately this is all I\\'m able to see:\\n\\n\\n\\n\\n\\n\\nSign #249  urlLink CNN  needs some sense slapped into them:  it\\'s the day after an election, Republicans have taken control the Senate, Dick Gephardt is stepping down as House minority leader and right now, on Larry King, they\\'re talking about the Winona Ryder verdict.  At least they took down the  BREAKING NEWS  graphic.\\n\\n\\n\\n\\n\\n\\nThere\\'s a  urlLink new trailer  for Steven Soderbergh\\'s  Solaris  up today -- one of the movies I\\'m most looking forward this fall.  I got a copy of the great DVD release of Tarkovsky\\'s  urlLink original film  from  urlLink The Russian Cinema Council  a short while ago.  Which I hope to review for Mindjack before the new film\\'s released.\\n\\n\\n\\n\\n\\n\\nCory Doctorow  urlLink reviews Smart Mobs  in the new Mindjack:    Smart Mobs are the Slashdot effect applied to the meatspace zeitgeist. A squillion like-minded souls who don\\'t know each other and will never meet pop out of the transmetropolitan brickface and break the white-noise balance of atomic viewpoints to speak with one voice, roaring a righteous YES or an adamant NO without organizers, without leaders, without manifestoes or forethought.   Also this issue, an in-depth look at  urlLink the Internet Archive  by Doug Roberts.   Tucked away in one of the seediest neighborhoods of San Francisco is a roomful of over two hundred computers with a terabyte of data stored on every three. Stairs from the street lead up an intimidating hallway that opens into a room with 15-foot ceilings and just-this-side of hip ductwork in the ceiling. To the right is a storage area with a single desk, to the left are Baker\\'s racks tightly packed with off-the-shelf HP desktop machines, each turned on it\\'s side to maximize the space. Somewhere in all that ductwork, a fan is squeaking painfully. Walking into this echoey, over-warm warehouse space, it\\'s easy to be underwhelmed until you realize what you are looking at: spinning away on these computers is nothing less than a copy of the Internet from 1996 until today.\\n\\n\\n\\n\\n\\n\\nMindjack is back after a short break.  The new issue has  urlLink a report  on the Supernova conference by Doug Roberts.  Supernova 2002 was an ambitious conference and Kevin Warbach, the organizer along with pulver.com, acknowledged that from the outset when he said that he had crammed a three day conference into two days. The conference was touted as a glimpse into the decentralized future. Decentralized systems, as may be obvious, are systems that have no single point of control - the Internet is a perfect example. No one entity or agency controls it, no single server provides the service, and no single point of failure will bring down the system. Warbach sees these types of systems as the fundamental issue in computing for the next decade. By then end of day one, however, it became clear that the conference was going to be about one thing: blogging.\\n\\n\\n\\n\\n\\n\\nI just discovered through  urlLink Smartmobs.com  that Clive Thompson has a  urlLink blog .  It\\'s great.  Which reminds me, I really need to update the blogroll on this site.\\n\\n\\n\\n\\n\\n\\nDoug Roberts is covering  urlLink Supernova  for Mindjack and  urlLink blogging it live .\\n\\n\\n\\n\\n\\n\\nSometimes Amazon.com is just plain eerie.\\n\\n\\n\\n\\n\\n\\nIf you\\'re going to be in Los Angeles on February 15, don\\'t miss  urlLink Live from the Blogosphere , a panel discussion to be put on by Rhizome.LA and moderated by Xeni Jardin.  \"Live from the Blogosphere!\" brings together six innovators in blogging: Mark Frauenfelder, Heather Havrilesky, Evan Williams, Susannah Breslin, Doc Searls, and Tony Pierce. The panel will discuss the birth of blogging, the emergent tension between blogs and traditional journalism, innovations in blogging such as video-blogging, audio-blogging, and mobile-blogging, the shifting roles of race and gender in the Blogosphere, the state of the blog economy, and the way blogs may be reshaping contemporary media.   (cross-posted from  urlLink Daily Relay )\\n\\n\\n\\n\\n\\n\\nI review a cool laptop bag today in  urlLink Daily Relay .\\n\\n\\n\\n\\n\\n\\nDouglas Rushkoff meets Al Gore and is  urlLink suitably impressed .   He was a much brighter, open-minded, and - if I might add - cosmic thinker than I gave him credit for, before. I felt like he was really one of \"us,\" if you know what I mean: the kids in college who got those wild thoughts about how everything in the world, and even beyond, is connected somehow. And evolving. Only Gore found this organic view of reality reflected in the Constitution of the United States, and sought to enable the extension of this Enlightenment thinking into practical reality.\\n\\n\\n\\n\\n\\n\\nNew Mindjack up today!  Cory Doctorow on  urlLink William Gibson .  Justin Hall on  urlLink Super Monkey Ball 2 .\\n\\n\\n\\n\\n\\n\\nCheck out  urlLink this preview  of Warren Ellis and Colleen Doran\\'s new graphic novel,  urlLink Orbiter .\\n\\n\\n\\n\\n\\n\\nAh crap.   urlLink Shift magazine publishes its last issue .  Shift has pulled out from dire situations before, but this looks to be the end, which is a real shame.  They\\'ve been a consistently great magazine and an important voice in the ever-changing sphere of digital culture.  The final issue is due out in two weeks, I urge everyone to pick up a copy.\\n\\n\\n\\n\\n\\n\\nMindjack\\'s been  slashdotted .\\n\\n\\n\\n\\n\\n\\nurlLink The Fray  redesigned for the first time in over six years.  Blog coming soon.\\n\\n\\n\\n\\n\\n\\nurlLink Deepsky  have posted their remix of David Bowie\\'s \"I Took a Trip an a Gemini Spaceship\" on their website.  Not as good as the original, but worth a listen.   urlLink RealAudio link\\n\\n\\n\\n\\n\\n\\nurlLink This post  by Warren Ellis to his blog isn\\'t just cool because he linked to Mindjack:  At Bruce Sterling\\'s house in Austin, Texas, Howard Waldrop beams Cory Doctorow an mp3 of some swampy old rock\\'n\\'roll from the dawn of time. The next day, across town, his powerbook scorching the flesh off his legs, he beams it over to me in England. I put it up on a private server for Fraction in Kansas City and Laurenn in San Francisco, while reading the first few chapters of Cory\\'s new book, which he sent to me from an airport a few days before, squatting by the power outlet next to the public toilets.  urlLink continued...\\n\\n\\n\\n\\n\\n\\nSlashdot launches  urlLink games.slashdot.org  and, holy cow, it\\'s ugly.\\n\\n\\n\\n\\n\\n\\nMindjack contributor and game reviewer, Justin Hall, has written a great article on  urlLink ethics in video game journalism  for the Online Journalism Review.  The first print magazine covering video games Electronic Games was co-founded by Bill Kunkel in 1981.  Kunkel describes those early days in a recent interview: \\x93To an extent, we were cheerleaders for the industry -- we loved these games, we wanted to see more of them, we wanted to keep writing about them.\\x94  Not much has changed in the past 20 years.  Game publications and Web sites still mostly employ low-paid hobbyists who are easy targets of lavish marketing events that encourage inappropriate ties between game makers and game critics.\\n\\n\\n\\n\\n\\n\\nThis is incredible.  Scientists have discovered a \"Colossal\" squid in the Antartic, much bigger than the previous biggest giant squid found.  And it\\'s not even fully grown.  Deep sea exploration is nearly as fascinating to me as space exploration.  Just imagine that there\\'s something like this that no one has ever seen alive.  It\\'s interesting that the article notes the funding for the research is from the Discovery Channel.  While it\\'s great there\\'s some funding available, just as with space exploration, there should be much more.\\n\\n\\n\\n\\n\\n\\nGoogling for sites that mention Mindjack, other than mindjack.com, suprisingly results in 5,800 links.  I like the description on  urlLink this one .  Mindjack Magazine. One pusle showing the osmotic swelling of Digital culture as the norm.\\n\\n\\n\\n\\n\\n\\nNew Mindjack up today.   urlLink     Josh examines the online, interconnected groups of people that you turn to for advice on music, art, fashion, books, etc., and the broader implications of these taste tribes.  Also, Justin Hall  urlLink reviews Panzer Dragoon Orta  for Xbox.  The illustration, btw, is by  urlLink Matt Hinrichs , who will hopefully be doing semi-regular ones for us.\\n\\n\\n\\n\\n\\n\\nI review  urlLink Soul Calibur 2, Syberia, and Post Mortem  in the new Mindjack.\\n\\n\\n\\n\\n\\n\\nVia  urlLink scrubbles.net : a (nearly) complete index of  urlLink Alfred Hitchcock\\'s cameo\\'s .\\n\\n\\n\\n\\n\\n\\nMy article,  urlLink Reunderstanding Movies , about how DVDs and the Internet are changing the way we watch movies, is in the new  urlLink Mindjack .\\n\\n\\n\\n\\n\\n\\nI\\'ve started a film blog:   urlLink melanson.ca/movies .  No promises on how often I\\'ll post to it, but I try to do as many reviews as I can.\\n\\n\\n\\n\\n\\n\\nFor your linking pleasure, a new Mindjack ad.  More sizes coming soon.\\n\\n\\n\\n\\n\\n\\nI\\'ve been blogging more at  urlLink Mindjack\\'s Daily Relay  lately and hope to be able to keep it up.  We\\'ve got lots of good stuff coming up in Mindjack in the next few months, so keep your eye on the blog for further updates.\\n\\n\\n\\n\\n\\n\\nApparently from a sidewalk in Austin.\\n\\n\\n\\n\\n\\n\\nDerek Powazek\\'s comment over in  urlLink this entry  at Blackbeltjones reminded me of Amazon.com\\'s incredibly good  urlLink Friends & Favorites  area.  Mr. Bezos, if you\\'re reading this, please add this to  urlLink Amazon.ca .  Canadians have friends too.\\n\\n\\n\\n\\n\\n\\nI\\'ve been playing around with  urlLink Flickr  lately, the latest toy from the mad geniuses at  urlLink Ludicorp . You can find my Flickr profile  urlLink here .  I\\'ve also started a  urlLink Mindjack group  as a bit of an experiment.  If it works out I may try to integrate it into the site.\\n\\n\\n\\n\\n\\n\\nNow in Mindjack:  urlLink Is Nothing Sacred? Digital Music for a Digital Age  by Ian Dawe. Antero Alli\\'s  urlLink Hysteria  reviewed on DVD by Jesse Walker.\\n\\n\\n\\n\\n\\n\\nurlLink     \\t\\t\\t \\t\\t\\t\\t \\t\\t\\t urlLink 1250_img0028 \\t\\t\\t \\t\\t\\tOriginally uploaded by \\t\\t\\t urlLink QuizMaster2000 .\\t\\t\\t \\t\\t \\t      urlLink    Posted by  urlLink  Don Melanson   from  urlLink Flickr .    urlLink\\n\\n\\n\\n\\n\\n\\nThe new issue of Mindjack is  urlLink now online . In this issue:  urlLink Linked Out: blogging, equality and the future  by Melanie McBride. Plus  urlLink Kill Bill Vol. 2  reviewed by Jesse Walker.\\n\\n\\n\\n\\n\\n\\nThere\\'s a  urlLink new issue  of Mindjack now online.  In it, the first article from Mindjack\\'s newest contributor J.D. Lasica; he writes about copyright law in  urlLink \"The Killing Fields\" .  Also, I review DVDs of  urlLink Breathless, Russian Ark, and Z .  Spread the word.\\n\\n\\n\\n\\n\\n\\n\"An idea is a funny thing.  It does nothing unless you do.\"\\n\\n\\n\\n\\n\\n\\nI\\'m looking for someone to be Advertising Director for Mindjack. Must have advertising sales experience and be willing to work on commission.  This is a contract job that can be done from home.  If you\\'re interested, please email:  urlLink donald@mindjack.com  (no attachments).   Also, we\\'re constantly looking for great new writers.  If you want to write for us, drop an email.\\n\\n\\n\\n\\n\\n\\nI\\'m pleased to announce that VRML creator and all-around visionary Mark Pesce has joined Mindjack\\'s ever-growing stable of writers.  His first article,   urlLink Redefining Television  , is an insightful examination of the state and future of the medium.  Here\\'s a snippet:   In the earliest days of television, writers like George Orwell in  1984  and Ray Bradbury in  Fahrenheit 451  projected television as the instrumentality of a totalitarian future - a monolithic entity dispensing propaganda. And, if any of you occasionally watch Fox News, you can see they weren\\'t that far off the mark. But here\\'s the thing: the monolithic days of television are numbered. Actually, they\\'ve already passed - though, as yet, very few people realize this.  To understand why, we need to go back to first principles. What is television? Here\\'s a functional definition:   Television is the capture, encoding, transmission, reception, decoding and display of moving images.   This definition applies to the Golden Age of television, and to the present era. Yet the definition of every word in the definition of television has changed, because of the introduction of digital production, encoding, transmission, reception, decoding and display technologies.\\n\\n\\n\\n\\n\\n\\nI\\'m using this blog to play around with  urlLink Blogger\\'s new features .  It will return to normal shortly.\\n\\n\\n\\n\\n\\n\\nI\\'m blogging a lot more on my  urlLink film blog  than I am here.  If you like movies, check it out.  And, of course, I continue to write and blog at  urlLink Mindjack .\\n\\n\\n\\n\\n\\n\\nurlLink GC-red   Revisar en detalle la propuesta de GC-red. Aparenta ser una comunidad interesante, dedicada a la formaci\u00f3n de gestores del conocimiento.\\n\\n\\n\\n\\n\\n\\nurlLink Good Practices for Managing Knowledge - 14 Aug 2003 : \"Good Practices for Managing Knowledge\"  Revisar este paper, se habla de bases de conocimiento, paginas amarillas como directorios y comunidades.\\n\\n\\n\\n\\n\\n\\nurlLink Introducing Patterns\\n\\n\\n\\n\\n\\n\\nurlLink Dale H. Emery \\x97 Conversations with Dale \\x97 \"I Don\\'t Have Time\"\\n\\n\\n\\n\\n\\n\\nurlLink The Satir Change Model \u00a9 2000 Steven M. Smith   Improvement is always possible. This conviction is the heart of the transformation system developed by family therapist Virginia Satir. Her system helps people improve their lives by transforming the way they see and express themselves.\\n\\n\\n\\n\\n\\n\\nurlLink Christian Sepulveda\\'s Blog: Emotional Resistance to Agile Development\\n\\n\\n\\n\\n\\n\\nurlLink Human Body Version 2.0   In the coming decades, a radical upgrading of our body\\'s physical and mental systems, already underway, will use nanobots to augment and ultimately replace our organs. We already know how to prevent most degenerative disease through nutrition and supplementation; this will be a bridge to the emerging biotechnology revolution, which in turn will be a bridge to the nanotechnology revolution. By 2030, reverse-engineering of the human brain will have been completed and nonbiological intelligence will merge with our biological brains.\\n\\n\\n\\n\\n\\n\\nurlLink Tesugen, Peter Lindberg\\x92s Weblog/Blog\\n\\n\\n\\n\\n\\n\\nurlLink Jeremy Hiebert\\'s headspaceJ -- Instructional Design and Technology\\n\\n\\n\\n\\n\\n\\nurlLink Jeremy Hiebert\\'s headspaceJ -- Instructional Design and Technology   \"People will object to games that have a variety of choices because they can\\'t limit the choices their children make,\" Gee said. \"However, if you remove that type of ambiguity, you\\'ve removed any sense of morality from the game because there are no consequences to bad decisions.\"\\n\\n\\n\\n\\n\\n\\nurlLink Dale H. Emery \\x97 Articles\\n\\n\\n\\n\\n\\n\\nurlLink Dale H. Emery \\x97 Managing Yourself Through Change   Major change is difficult. Unexpected change is even more difficult. In this paper, I offer some possibilities for navigating change more effectively.  In this paper, Dale H. Emery present a model of change, which describes the stages we go through as we respond to change. Next he look at why each stage happens, why we respond the way we do, and how we move from one stage to the next. Then he offer some ideas about how to respond more effectively in each stage.\\n\\n\\n\\n\\n\\n\\nurlLink Open Directory - Computers: Software: Project Management: Open Source\\n\\n\\n\\n\\n\\n\\nurlLink Articles and Insights | PM Solutions\\n\\n\\n\\n\\n\\n\\nurlLink The Rational Edge -- September 2003 -- Agile RUP for non-object-oriented projects   The Unified Modeling Language (UML) and IBM Rational Unified Process,\u00ae or RUP,\u00ae are standard fixtures on most object-oriented projects today. UML is a notation for visually expressing object-oriented concepts; RUP is a process framework that uses the UML to express the content of its artifacts and tasks. But what do RUP and the UML offer that portion of our software community doing non-object oriented development? This article is my answer to that question. As we shall see, RUP and UML can be used quite effectively on software projects in COBOL, C, or even (the dreaded) assembler language.\\n\\n\\n\\n\\n\\n\\nurlLink ALLPM - The Project Managers HomePage :: The Project Managers HomePage\\n\\n\\n\\n\\n\\n\\nurlLink The PMFORUM - For Advancement Project Management Learning : \"PMFORUM\u00ae provides instant  access  to world-wide project management information.\"\\n\\n\\n\\n\\n\\n\\nurlLink Max Wideman\\'s Musings Web Log : \"Max Wideman is an engineer and professional project manager with experience in systems, social and environmental projects, as well as design and engineering projects. He is a Fellow of the Project Management Institute He is also a Fellow of the Institution of Civil Engineers (UK), the Engineering Institute of Canada, and the Canadian Society of Civil Engineering. His web site is at http://www.maxwideman.com \"\\n\\n\\n\\n\\n\\n\\nurlLink Project Management World Today May 2003 - Project Management Community\\n\\n\\n\\n\\n\\n\\nurlLink NYSFIRM | Groups | NYS Project Management Community of Practice   The NYS PM Community of Practice provides a forum for an interactive exchange of ideas, practices, lessons learned, etc. between project managers in New York government. The group meets on a quarterly basis and is open to all interested in learning more about PM topics. All meetings are posted on the Forum\\'s Calendar of Events.\\n\\n\\n\\n\\n\\n\\nurlLink Project Management World Today February/March 03 - Education and Careers   The author describes three characteristics of weblogs that make them suitable for project teams:   Easy, fast, and continuous communication  Personalisation and identification  Automatic storage of knowledge   He argues that virtual teams must overcome physical separation. All that makes sense. But there\\'s more too it than that. Project teams need to be in charge of their projects. They need confidence while exercising judgement. The p-log just might be the tool.\\n\\n\\n\\n\\n\\n\\nurlLink Proposal for a P-Log Specification   A Project Team Tool for Lean/Agile Project Methodologies\\n\\n\\n\\n\\n\\n\\nurlLink Scrum Overview   Scrum is an iterative, incremental process for developing any product or managing any work. It produces a potentially shippable set of functionality at the end of every iteration\\n\\n\\n\\n\\n\\n\\nurlLink elearnspace. everything elearning.   Learner-centered, lifelong learning has been the cry of knowledge society visionaries for the last decade. Yet learning continues to be delivered with teacher-centric tools in a twelve week format. Society is changing. Learners needs are changing. The course, as a model for learning, is being challenged by communities and networks, which are better able to attend to the varied characteristics of the learning process by using multiple approaches, orchestrated within a learning ecology.\\n\\n\\n\\n\\n\\n\\nurlLink Free Range Learning Support   When you were three or four years old, you asked a lot of questions about the world you experienced.  Why is the sky blue?  Do fish drink?  Why does the moon follow us?  Given your developing command of language and curiosity, it was an easy and rewarding way to augment your experience of the world. By the time you reached six or seven years old, you probably stopped asking so many questions about the things around you. Perhaps you realized that the endless stream of questions began to annoy more than charm your parents. Perhaps you learned how to find the answers yourself. Perhaps, as many learning theorists suggest, your schooling had acclimated you to posing \"appropriate\" questions and giving \"correct\" answers.  In my master\\x92s project thesis, I hope to reawaken and re-enable this spirit of inquiry and engagement in the world for adult learners, through the development of an interconnected set of services I call free range learning support.\\n\\n\\n\\n\\n\\n\\nurlLink NetworkMoves\\x97The Monthly Newsletter Highlighting Career and Business Networking Ideas   The ready availability of information on the Internet makes it easy for people to do free background checks on people before entering into business relationships. You should definitely do this before you hire an employee, consultant, or contractor.\\n\\n\\n\\n\\n\\n\\nurlLink HR.com Human Resources Management, Training, Jobs, Information   The Google Search engine has become the Swiss Army Knife of the Internet \\x96 it has more uses than finding relevant web pages.    Some of the uses are infringing on the domain of Human Resources. It is easy for a boss to do a background check on a potential new employee. With Google, the potential new hire can turn the tables, and do a pretty decent background check on the new boss!   Google\\x92s greatest application as an \\x91HR tool\\x92 is not in the corporation \\x96 it is out among the free agents, consultants and entrepreneurs who live and work by reputation and experience. It is here where \\x91Google numbers\\x92 become very important.   Mine Is Bigger Than Yours\\n\\n\\n\\n\\n\\n\\nurlLink scriptygoddess\\n\\n\\n\\n\\n\\n\\nurlLink Seb\\'s Open Research\\n\\n\\n\\n\\n\\n\\nurlLink HBS Working Knowledge: Career Effectiveness: Speechwriting Under the Gun   It doesn\\x92t matter to your audience if you have ten days or ten minutes to write a speech. You still must deliver. Here are tips for speeding your speech prep.\\n\\n\\n\\n\\n\\n\\nurlLink The McKinsey Quarterly: The psychology of change management   Companies can transform the attitudes and behavior of their employees by applying psychological breakthroughs that explain why people think and act as they do.\\n\\n\\n\\n\\n\\n\\nurlLink SPIDER:\\xa0 Social Psychology of Information Diffusion -- Educational Resources   a website of links and article/book reviews (below) relating to the spread of culture, information, and even diseases, maintained by Alan Reifman.\\n\\n\\n\\n\\n\\n\\nurlLink HR.com Human Resources Management, Training, Jobs, Information\\n\\n\\n\\n\\n\\n\\nurlLink KAYWA AG\\n\\n\\n\\n\\n\\n\\nurlLink BlogPlanet\\n\\n\\n\\n\\n\\n\\nurlLink Azure - a free blog client for your phone and PDA\\n\\n\\n\\n\\n\\n\\nurlLink Reed\\x92s Law and Communities   Reed\\x92s Law and Communities \\x96 or Why Yahoo Groups, Mailing Lists and Forums Don\\x92t Work\\n\\n\\n\\n\\n\\n\\nurlLink Joi Ito\\'s Moblogging, Blogmapping and Moblogmapping related resources as of6/10/2003\\n\\n\\n\\n\\n\\n\\nurlLink Fast Company | The Blog of Things To Come\\n\\n\\n\\n\\n\\n\\nurlLink Short Guide to Consensus Building   Cuando en una comunidad es necesario lograr consenso y no solo una simple mayoria, esta guia puede ser de utilidad.\\n\\n\\n\\n\\n\\n\\nurlLink IST Project Templates\\n\\n\\n\\n\\n\\n\\nurlLink rebelpixel productions \u00bb wp\\x96recent\\x96links\\n\\n\\n\\n\\n\\n\\nurlLink Unofficial Wordpress Support Forums - powered by vBulletin\\n\\n\\n\\n\\n\\n\\nurlLink openMDX - the leading open source MDA platform   openMDX is the leading open source MDA platform based on the Object Management Group\\x92s Model Driven Architecture standards. openMDX supports J2SE, J2EE, and .NET.  openMDX enables software architects and developers to build and integrate software applications in a highly automated and industrialized way.\\n\\n\\n\\n\\n\\n\\nurlLink Penis owner\\'s manual   Welcome to the world-wide family of penis owners. We trust that you will enjoy many years of trouble-free service from yours. To help ensure that you will, we encourage you to familiarise yourself with the following equipment descriptions, operating instructions and maintenance requirements.\\n\\n\\n\\n\\n\\n\\nurlLink Knowledge@Wharton\\n\\n\\n\\n\\n\\n\\nurlLink ARP - Sociedad para el Avance del Pensamiento Cr?tico\\n\\n\\n\\n\\n\\n\\nKnowledge Communities in Japan  Like Fred, I also found Hideo Yamazaki\\x92s paper, \"Knowledge Communities  in Japan A Case Study,\" to be quite interesting.  However, I\\x92m not real  sure if I agree that such online or IT based knowledge communities are  real \\x93knowledge\\x94 networks that allow for knowledge building, or rather  if they are actually more to the tune of \\x93information\\x94 centers.  Velocity - the speed with which knowledge moves through an organization  (1)  Viscosity - the richness or thickness of the knowledge transferred (1)  Mobil Oil\\'s engineers developed some sophisticated ways of determining  how much steam is required to drill under certain conditions.  When  they applied their techniques, they found they could reduce the amount  of steam they generated themselves and bought from outside sources.   Since they knew the precise amount needed, the potential savings were  huge.  So they embedded the technique in an intelligent system, with  the main focus being the knowledge\\'s velocity in order to quickly get  the information out to the field.  This was done by sending a memo to  all the drilling operations detailing the calculations and benefits.   They assumed other sites would quickly adapt the innovation.  Nothing  happened.  The effective viscosity level was zero.  Simply improving a process will not be enough to win over everyone.  So  Mobil came up with some other techniques for transferring the  knowledge, such as videos and case studies and soon raised the adoption  rate to 30%.  It now looks as if it will grow to 50%.  Will it ever  reach 100%?  Who knows?  The resistance to abandoning processes that  have been successful for years is a universal phenomenon and is not  just limited to Mobil.  Velocity strips knowledge and information down to its bare essentials  by removing portions of its context and richness.  And this stripping  effect often takes the message down to the next level for its intended  receivers-- from knowledge to information or from information to data.   Thus the intended knowledge exchange fails.  Now look at how 3M does it -- they have regular meetings and fairs for  exchanging knowledge.  One of their most famous products, scotch tape,  was invented by Dick Drew...a sandpaper salesman.  In almost any other  company his idea would have been tossed out as tape and research were  not his specialty, yet the culture of 3M allows for the \"viscosity\" of  information to be spread.  And the way they ensure it spreads is  through meetings and knowledge fairs -- it is not left for a chance  happening on their intranet.  In the article, Professor Yamazaki writes, \\x93High-context based  communities in many Japanese companies have been gradually crumbling.\\x94   Perhaps this might be part of Japan\\'s problems -- the replacement of  communities where knowledge exchanges can likely occur with online  communities that often work better as information exchanges.  No don\\'t  get me wrong, both knowledge and information exchanges can occur in  either type of community, however, it is easier to exchange knowledge  in certain types of environments.  Being online allows one to gather a lot of information, yet the vary  nature of it \\x96 speed, works against it as being a knowledge-enabler.   Knowledge requires rich sources and context.  For example, one of the  best knowledge-enablers for a beginning learner is apprenticeship as it  allows a rich exchange of concepts and ideas between the learner and  teacher.  As one starts to slowly move away from this form of learning,  the knowledge exchange slowly starts to decrease, while the rate of  information flow increases.  That is, in a class, the information is  going out to a number of learners at once, hence the speed of  information exchange increases; however, the flip-side is that  one-on-one interactions decrease, thus the rate of knowledge transfer  decreases.  This is the first dimension of the  data/information/knowledge continuum \\x96 Viscosity/Velocity.  The second dimension of the data/information/knowledge continuum is the  number of paths or streams of information flow.  A single email  (monophonic) sent to a direct report is one stream, where as this list  normally has multiple posts (context) to each thread.  The more posts,  then the more viewpoints one can gather or harvest.  Once a learner  moves past the beginner stage, then these multiple viewpoints start to  become invaluable as they add context to the knowledge base that one  has gained.  For example, A PhD student studies one small section of  his or her field, but does so from multiple contexts.  There is a diagram showing these various relations at: http://www.nwlink.com/~donclark/km/knowledge2.jpg  Throughout the various Knowledge Communities and Communities of  Practice, there are different levels of information and knowledge  exchanges.  So how does one determine if the community is mainly an  information exchange or a knowledge creator or builder?  What practices  have to be put in place to raise a information exchange to a knowledge  exchange?  For example, in Professor Yamazaki\\'s PowerPoint slide, there  is a facilitator for helping with the \"stroking\" of the members.  Is  this enough to ensure the exchange of knowledge?  Another example is  this list -- when does it help to build or exchange knowledge rather  that just exchange information?  Must there be participation or can one  just read the postings and become knowledgeable?  Or are they simply  becoming \"informed\\'?  NOTES 1. Thomas H. Davenport and Laurence Prusak (2000). \"Working Knowledge.\"  Cheers, and thanks for the excellent article! Donald Clark  ~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~  Big Dog\\'s Bowl of Biscuits http://www.nwlink.com/~donclark  ~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~\\n\\n\\n\\n\\n\\n\\nurlLink Big Dog\\'s Bowl of Biscuits : \"A door is what a dog is perpetually on the wrong side of.\"\\n\\n\\n\\n\\n\\n\\nurlLink dasBlog.net   Newtelligence dasBlog, a free, BSD-licensed weblog engine for the .NET Framework provided by newtelligence AG. dasBlog is an evolution of the BlogX weblog engine initially written by Chris Anderson and contributors\\n\\n\\n\\n\\n\\n\\nurlLink TimothyHumphrey.WebControls   TimothyHumphrey.WebControls is a collection of web controls for use with ASP.NET applications. The controls have various browser requirements but in general they are made to look best in modern browsers such as Internet Explorer 5+ and Gecko-based browsers such as Mozilla or Netscape 7.\\n\\n\\n\\n\\n\\n\\nurlLink FreeTextBox 2.0   ASP.NET HTML editor control for IE and Mozilla. By John Dyer.\\n\\n\\n\\n\\n\\n\\nurlLink Roaming charges: Packets to the people!\\n\\n\\n\\n\\n\\n\\nurlLink Roaming charges: Packets to the people!\\n\\n\\n\\n\\n\\n\\nurlLink www.TxOutcome.Org Home of OIO: Open Infrastructure for Outcomes   Open Infrastructure for Outcomes (OIO) system facilitates the creation of flexible and portable patient/research records. It aims to achieve the \" Holy Grail \" of data portablity as elegantly described by John G. Faughnan.   The major components of the OIO system are the web-accessible OIO Server and OIO Library. OIO Server is a highly flexible web-based data management system that manages users, patients, and information about patients. (Of course, the same system can just as easily manage information about customers, invoices, shipments, and accounts.)   OIO Library is a metadata repository that facilitates the sharing of metadata (e.g. plug-and-play web-forms, project descriptions) between users and between OIO Servers. The OIO Library also hosts a public/open content database of open source medical software projects and related documents.\\n\\n\\n\\n\\n\\n\\nurlLink In Search of the Perfect Causal Loop Variable\\n\\n\\n\\n\\n\\n\\nurlLink TOP 10 OPEN SOURCE TOOLS TO SPREAD YOUR MESSAGES | Digital Squeeze\\n\\n\\n\\n\\n\\n\\nurlLink CWXML/BXML Home Page\\n\\n\\n\\n\\n\\n\\nurlLink 1060.org - 1060 NetKernel Home\\n\\n\\n\\n\\n\\n\\nurlLink Links to Open Source Health Care Resources : \"Electronic Medical Records system \"\\n\\n\\n\\n\\n\\n\\nurlLink Pennington Firm - Linux and Open Source Consulting\\n\\n\\n\\n\\n\\n\\nurlLink ATCRM - Open Source CRM ::\\n\\n\\n\\n\\n\\n\\nurlLink OpenEMR.net :: Open Source Electronic Medical Records\\n\\n\\n\\n\\n\\n\\nurlLink GNU/Linux And Open Source Medical Software News\\n\\n\\n\\n\\n\\n\\nurlLink OpenEMR 2.0.0 Release\\n\\n\\n\\n\\n\\n\\nurlLink UMD News: February 18, 2003   UMD Announces Move to \"Open Source\" Innovative, New Electronic Portfolio\\n\\n\\n\\n\\n\\n\\nurlLink XChart: easier than paper\\n\\n\\n\\n\\n\\n\\nurlLink Information Axioms - Nautilus Institute\\n\\n\\n\\n\\n\\n\\nurlLink Seb\\'s Open Research\\n\\n\\n\\n\\n\\n\\nurlLink Forbes.com: Pix Populi\\n\\n\\n\\n\\n\\n\\nurlLink ChristianLindholm.com\\n\\n\\n\\n\\n\\n\\nurlLink Azureus : Java BitTorrent Client   It provides a bittorrent protocol implementation using java language.  Azureus offers multiple torrent downloads, queuing/priority systems (on torrents and files), start/stop seeding options and instant access to numerous pieces of information about your torrents. Azureus now features an embedded tracker easily set up and ready to use.\\n\\n\\n\\n\\n\\n\\nurlLink A Beginner\\'s Crash Course into Object Oriented Programming - OSNews.com\\n\\n\\n\\n\\n\\n\\nurlLink Poseidon for UML - by Gentleware, just model - Products   Poseidon for UML is a popular, fully-fledged UML CASE tool. It evolved from the open-source project ArgoUML and has turned it into a world class modeling tool. Today, it has the fastest growing user community and is famous for its superior usability. Poseidon for UML is delivered in several editions to meet the needs of different users.\\n\\n\\n\\n\\n\\n\\nurlLink argouml.tigris.org  ArgoUML is a modelling tool that helps you do your design using UML.\\n\\n\\n\\n\\n\\n\\nurlLink PyDBDesigner - an entity-relation modeling tool   PyDBDesigner is a free relational database modeling tool that is planned to be able to design both logical and physical models similarly to Erwin(c) and PowerDesigner. Its goal is to provide a useful toolset that allows developers analysts and database designers to design and develop databases from the verbal model to the schemas and SQL scripts.\\n\\n\\n\\n\\n\\n\\nurlLink Download2Home\\n\\n\\n\\n\\n\\n\\nurlLink EasyMoblog - an open source blog and moblog platform\\n\\n\\n\\n\\n\\n\\nurlLink mono\\n\\n\\n\\n\\n\\n\\nurlLink Binary Bonsai\\n\\n\\n\\n\\n\\n\\nurlLink Portada - Wikipedia\\n\\n\\n\\n\\n\\n\\nurlLink phpMyFAQ - open source FAQ system for PHP and MySQL | welcome\\n\\n\\n\\n\\n\\n\\n\"Estar juntos es un comienzo.  Mantenernos juntos, un progreso.  Trabajar juntos, es un \u00e9xito.\"  -Henry Ford-\\n\\n\\n\\n\\n\\n\\nurlLink World Wide Web Research Tools\\n\\n\\n\\n\\n\\n\\nurlLink Minimalist Queue Services (MQS)   MQS  (Minimalist Queue Services)  is a  minimalist queueing  system in Free Software. The  main purpose is to provide a  set of basic methods in order to offer asynchronous messaging for program/application.   Applications  can use  this type  of services  in order  to  store and retrieve  message to  dedicated  queue in  the  queue services.   This permits to avoid direct connection between applications and provides a simple asynchronous communication system.   The MQS  Daemon provides a simple  set of XML-RPC  methods for pushing and getting  messages in a specific  queue (so you can  use the system with any application that can  talk XML-RPC). The storing of the queue is in a database (e.g. Mysql, Postgresql...) or in dedicated file(s).\\n\\n\\n\\n\\n\\n\\nurlLink GanttProject - Home   GanttProject 1.9.11 Release!\\n\\n\\n\\n\\n\\n\\nurlLink StickyMinds.com : Column Archive\\n\\n\\n\\n\\n\\n\\nurlLink Weblogs, Portals, and KM | MySmartChannels\\n\\n\\n\\n\\n\\n\\nurlLink Reiter\\'s Camera Phone Report\\n\\n\\n\\n\\n\\n\\nurlLink Home page Intec Telecom Systems\\n\\n\\n\\n\\n\\n\\nurlLink Color scheme\\n\\n\\n\\n\\n\\n\\nurlLink The Virtues of Chitchat - Making I.T. Work - CIO Magazine May 15,2004\\n\\n\\n\\n\\n\\n\\nurlLink PIXELSCRIPTS | XHTML & CSS Valid Templates for Weblogs and Websites\\n\\n\\n\\n\\n\\n\\nurlLink Google Blog\\n\\n\\n\\n\\n\\n\\nurlLink Bricolage   Bricolage is such a CMS. It is built on Mason, and you can use it to build large Mason-powered sites as well as sites driven by other technologies.\\n\\n\\n\\n\\n\\n\\nurlLink Stopdesign | The New Blogger\\n\\n\\n\\n\\n\\n\\nurlLink STORYATWORK.COM - Home Page : \"GoldenFleece is an international community of practice devoted to storytelling in business and organizations.\"  GoldenFleece is an international community of practice devoted to storytelling in business and organizations.\\n\\n\\n\\n\\n\\n\\nurlLink metakappa - knowledge ecosystem blog    metakappa is a blog by Gino Tocchetti, the first italian knowledge ecosystem blog, focused on knowledge ecosystems.\\n\\n\\n\\n\\n\\n\\nurlLink Product Design Engineering Consultancy in Glasgow, Scotland   fearsomENGINE - a rapid response product design consultancy, fusing a flair for inventive industrial design with sound engineering knowledge and experience.\\n\\n\\n\\n\\n\\n\\nurlLink PROAGE - A Empresa - Apresenta??   A PROAGE \u00e9 uma empresa criada por engenheiros com o objetivo de levar \u00e0 pr\u00e1tica das empresas conhecimentos e experi\u00eancias que tornem a administra\u00e7\u00e3o de seus projetos e de suas opera\u00e7\u00f5es mais eficientes e eficazes.\\n\\n\\n\\n\\n\\n\\nurlLink RSSTop55 - Best Blog Directory And RSS Submission Sites - Robin Good\\' Sharewood Tidings   Where To Submit My RSS Feeds And Weblog URLs To Get More Exposure, Visibility And Reach\\n\\n\\n\\n\\n\\n\\nurlLink Ari Paparo Dot Com: Big List of Blog Search Engines\\n\\n\\n\\n\\n\\n\\nurlLink Telefon?a m?vil... ?que piensan los usuarios, que dicen,? que comentan en los foros de internet?   Un an\u00e1lisis de 15.000 mensajes en foros especializados en telefon\u00eda m\u00f3vil desvela que los usuarios centran sus conversaciones en la conectividad de sus terminales. La posibilidad de sacar fotos con el m\u00f3vil, grabar v\u00eddeo y reproducir mp3 y radio, es el segundo atributo m\u00e1s comentado con un 16,6% de los mensajes, mientras que Bluetooth es la tecnolog\u00eda de conectividad m\u00e1s valorada, por delante del cable para PC y de los infrarrojos\\n\\n\\n\\n\\n\\n\\nurlLink InteractionDesigners.com   Dise\u00f1adores de Interacci\u00f3n Estamos aqu\u00ed para crear un hogar para dise\u00f1adores de interacci\u00f3n --Desconocemos todav\u00eda qu\u00e9 clase de hogar ser\u00e1. Tanto si encontramos, como si construimos nuestro hogar, este deber\u00e1 resolver los temas y desaf\u00edos que muchos de nosotros enfrentamos en nuestros puestos de trabajo.\\n\\n\\n\\n\\n\\n\\nurlLink ITracker: Java Issue Tracking Software   ITracker is a Java J2EE issue tracking system designed to support multiple projects with independent user bases. It supports features such as multiple versions and project components, detailed histories, per project user permissions and multiple email notifications.\\n\\n\\n\\n\\n\\n\\nurlLink Create-A-Character   Create a southpark character\\n\\n\\n\\n\\n\\n\\nurlLink UML Tool - Model-Code-Deploy software development platform\\n\\n\\n\\n\\n\\n\\nurlLink ElectricMonk.nl : WTP   WTP is a webbased (php) ftp client / File manager.\\n\\n\\n\\n\\n\\n\\nurlLink poetic pollution ? scripts ? Perl ? UF Prime (Counter/Tracker)\\n\\n\\n\\n\\n\\n\\nurlLink SocialTwister.com\\n\\n\\n\\n\\n\\n\\nurlLink Welcome to formsPlayer - The XForms Toolkit\\n\\n\\n\\n\\n\\n\\nurlLink OTD One Time Download   One Time Download (OTD) is a Perl CGI script that allows customers/users to easily download different files by submitting different one time tickets. It features access and bandwidth control for each file.\\n\\n\\n\\n\\n\\n\\nurlLink oss.ipov.org   jSai is a servlet authentication implementation which aims to allow easier to use, more powerful authentication for Java servlets. It is aimed at servlet developers who want more than the servlet spec, but not as much as a full Enterprise Server. It provides a powerful set of authentication and authorization features for user level security in servlet-based Web applications. NTML HTTP Authentication is now supported via jCIFS\\n\\n\\n\\n\\n\\n\\nurlLink Synapse - Wikipedia, the free encyclopedia   Chemical synapses (\"synapses\") are specialized junctions through which cells of the nervous system signal to one another and to non-neuronal cells such as muscles or glands.  Synapses are circuits in which the neurons of the central nervous system interconnect. They are thus crucial to the biological computations that underlie perception and thought. They also provide the means through which the nervous system connects to and controls the other systems of the body.\\n\\n\\n\\n\\n\\n\\nurlLink Volere Requirements Specification Template   This requirements specification template is used as a basis for your specifications. The template provides sections for each of the requirements types appropriate to today\\'s software systems\\n\\n\\n\\n\\n\\n\\nurlLink StickyMinds.com : Better Software magazine : Collaborate for Quality   Project teams are searching for ways to develop requirements that are as free from defects as possible. Here\\'s how you can use collaborative workshops, along with walkthroughs and QA checklists, to develop high-quality requirements.\\n\\n\\n\\n\\n\\n\\nurlLink Sizing Software Using Testable Requirements   A sizing measure is fundamental to any software measurement program.  Software professionals need a sizing measure that is understood and accepted by software developers, users and executive management.  Client/server and web applications, in particular, demand better measurement tools to cope with the increasing complexity of the technology.  A practical, flexible, sizing measure is needed not only to develop more reliable estimates, but also to better communicate with technical and non-technical management on issues of scope, change and complexity.    The concept of testable requirements meets these needs.\\n\\n\\n\\n\\n\\n\\nurlLink John Blade\\'s Homepage ? wp-mail.php\\n\\n\\n\\n\\n\\n\\nurlLink SAP - SAP NetWeaver Home   The Open Integration and Application Platform Can your company adapt quickly enough to respond to new challenges or seize new opportunities? With SAP NetWeaver, it can. A comprehensive integration and application platform, SAP NetWeaver works with your existing IT infrastructure to enable and manage change.  With SAP NetWeaver, you can flexibly and rapidly design, build, implement, and execute new business strategies and processes. You can also drive innovation throughout your organization by combining existing systems while maintaining a sustainable cost structure.  SAP NetWeaver embraces Internet standards such as HTTP, XML, and Web services. Ensuring openness and interoperability with Microsoft .NET and Java 2 Platform Enterprise Edition (J2EE) environments such as IBM WebSphere.\\n\\n\\n\\n\\n\\n\\nurlLink WhoAt     \"Tell WhoAt where you are, or where you want to be, and we will tell you who is nearby. You\\'ll see pictures and profiles of people who match your dating profile. You can also keep a list of friends so you\\'ll know when they are next time you\\'re out on the town. Once you\\'ve found a match or a friend, WhoAt lets you chat anonymously until you decide to share information about yourself. WhoAt never tells strangers your exact location or any of your personal information. Read more about how we protect your privacy. WhoAt is designed to be used from your mobile phone, PDA, or computer. Find out more about WhoAt from your phone.\"\\n\\n\\n\\n\\n\\n\\nurlLink waste :: home   WASTE is an anonymous, secure, and encryped collaboration tool which allows users to both share ideas through the chat interface and share data through the download system. WASTE is RSA secured, and has been hearalded as the most secure P2P connection protocol currently in development.\\n\\n\\n\\n\\n\\n\\nurlLink Tucu\\'s Weblog   Argentino en SUN\\n\\n\\n\\n\\n\\n\\nurlLink Furl - Your web page filing cabinet   Furl is a tool for saving, sharing and finding information (you know, the pages you want to save for future reference but then can never find again). At the core, Furl is a place to store content you find on the Internet and know that you can find it again at any time in the future (i.e. articles, product descriptions, web sites, e-commerce receipts, etc.).   Furl is used for many purposes by many different types of people. From travel planning to recipe sharing. Medical research to sporting news. Thousands of people use Furl every day to save, share, and find information that is important to them.\\n\\n\\n\\n\\n\\n\\nurlLink rome: All feeds lead to Rome   ...because all feeds lead to Rome\\n\\n\\n\\n\\n\\n\\nurlLink Oddpost\\n\\n\\n\\n\\n\\n\\nurlLink Searching the BlogSphere\\n\\n\\n\\n\\n\\n\\nurlLink Geekzone, mobile forums\\n\\n\\n\\n\\n\\n\\nurlLink PieSpy - Inferring and Visualizing Social Network on IRC : \"PieSpy is an IRC bot that monitors a set of IRC channels. It uses a simple set of heuristics to infer relationships between pairs of users. These inferrences allow PieSpy to build a mathematical model of a social network for any channel. These social networks can be drawn and used to create animations of evolving social networks. \"\\n\\n\\n\\n\\n\\n\\nurlLink Nuclear Elephant: DSPAM    DSPAM (as in De-Spam) is an extremely scalable, open-source statistical hybrid anti-spam filter. While most commercial solutions only provide a mere 95% accuracy (1 error in 20), a majority of DSPAM users frequently see between 99.95% (1 error in 2000) all the way up to 99.991% (2 errors in 22,786). DSPAM is currently effective as both a server-side agent for UNIX email servers and a developer\\'s library for mail clients, other anti-spam tools, and similar projects requiring drop-in spam filtering. DSPAM has been implemented on many large and small scale systems with the largest systems being reported at about 125,000 mailboxes.\\n\\n\\n\\n\\n\\n\\nurlLink Sentinel TMS: Analysis and Visualization   \"To gain insights into the composition and dynamics of any network, it is important to have tools that create information from raw data. Advancements in the scientific disciplines of network analysis, social network analysis, and computer science have provided a variety of new tools to apply to the problems associated with terrorist networks.\"\\n\\n\\n\\n\\n\\n\\nurlLink Netvis Module - Dynamic Visualization of Social Networks   \"The NetVis Module is a free open source web-based tool to analyze and visualize social networks using data from csv files, online surveys, and dispersed teams. \"\\n\\n\\n\\n\\n\\n\\nurlLink Sipura Technology SPA-2000    \"Inexpensive, easy-to-install and simple-to-use, the SPA-2000 connects standard telephones and fax machines to IP-based data networks. IP telephony service providers and enterprises can offer users traditional and enhanced communication services via the customer\\'s broadband connection to the Internet or Local Area Network (LAN).\"\\n\\n\\n\\n\\n\\n\\nurlLink Cisco - Cisco ATA 186   The Cisco ATA 186 Analog Telephone Adaptor brings traditional telephones to the networked world. It is a handset-to-Ethernet adaptor that interfaces regular analog telephones with IP-based telephony networks. The Cisco ATA 186 is installed at the subscriber\\'s premises and supports two voice ports, each with its own independent telephone number\\n\\n\\n\\n\\n\\n\\nurlLink ONJava.com: SMS-Powered Applications\\n\\n\\n\\n\\n\\n\\nurlLink Open Source Applications Foundation   Our product (code-named \"Chandler\" after the great detective novelist Raymond Chandler,) is a Personal Information Manager (PIM) intended for use in everyday information and communication tasks, such as composing and reading email, managing an appointment calendar and keeping a contact list. Because of the ease with which Chandler users can share information with others, Chandler might be called the first Interpersonal Information Manager. (The term PIM was first used in conjunction with the product Lotus Agenda in the 1980\\'s. Chandler is the spiritual descendant of Agenda (and has a common designer in Mitch Kapor.)  Chandler is intended to be an open source personal information manager for email, calendars, contacts, tasks, and general information management, as well as a platform for developing information management applications. It is currently under development and will run on Windows, Mac, and Linux-based PC\\'s.\\n\\n\\n\\n\\n\\n\\nurlLink Trigenix - The Mobile Interface Company   Trigenix Ltd (formerly 3G LAB) was founded in March 2000 in Cambridge, UK. Since then, backed by venture capital investment from 3i, one of Europe\\'s largest venture capital groups, Trigenix has become an acknowledged leader in the development and deployment of user interfaces for mobile phones. In 2003 Trigenix was ranked 24th in the Sunday Times Fast Track 100, having grown revenue 700% since 2000.  The user interface is becoming a critical point for the uptake of data services. Trigenix can realise the huge revenue potential for operators in:   Scaling-up messaging and other data revenue  Gaining market share in download sales  Profiting from the desire for personalisation  Improving the User Experience    At the GSM Association Awards 2003, in February, Trigenix was recognised by the mobile industry body representing all GSM operators as Best Wireless Application Developer. The judging committee noted that Trigenix provides an elegant solution to an important problem for the mobile industry.\\n\\n\\n\\n\\n\\n\\nurlLink OpenSymphony - Welcome To OpenSymphony   \"OpenSymphony is an Open Source project dedicated to providing enterprise class J2EE components. Our components aim to stress simplicity, integration, pluggability and specification compliance. \"\\n\\n\\n\\n\\n\\n\\nurlLink Gentoo Linux Documentation -- Keychain   \"Many of us use the excellent OpenSSH as a secure, encrypted replacement for the venerable telnet and rsh commands. One of OpenSSH\\'s (and the commercial SSH2\\'s) more intriguing features is its ability to authenticate users using the RSA and DSA authentication protocols, which are based upon a pair of complementary numerical \\'keys\\'. And one of the main appeals of RSA and DSA authentication is the promise of being able to establish connections to remote systems without supplying a password. The keychain bash script makes handling RSA and DSA keys both convenient and secure. It acts as a front-end to ssh-agent, allowing you to easily have one long-running ssh-agent process per system, rather than per login session. This dramatically reduces the number of times you need to enter your passphrase from once per new login session to once every time your local machine is rebooted. \"\\n\\n\\n\\n\\n\\n\\nurlLink HAproxy Page   HAproxy is a high-performance and highly-robust TCP/HTTP load balancer which provides cookie-based persistence, automatic failover, header insertion, deletion, modification on the fly, advanced logging contents to help trouble-shooting buggy applications and/or networks, and a few other features. It uses its own state machine to achieve up to ten thousands hits per second on modern hardware, even with thousands simultaneous connections. It currently lacks SSL and keep-alive, both of which are planned.\\n\\n\\n\\n\\n\\n\\nurlLink apachefriends.org - xampp   \"Many people know from their own experience that it\\'s not easy to install an Apache web server and it gets harder if you want to add MySQL, PHP and Perl.  XAMPP is an easy to install Apache distribution containing MySQL, PHP and Perl. XAMPP is really very easy to install and to use - just download, extract and start\"\\n\\n\\n\\n\\n\\n\\nurlLink FeedBurner - The spark for syndication success\\n\\n\\n\\n\\n\\n\\nurlLink Full Circle Associates Online Interaction & Community Blog\\n\\n\\n\\n\\n\\n\\nurlLink BEA transforms mobile apps with Alchemy | CNET News.com\\n\\n\\n\\n\\n\\n\\nurlLink ZeroC Home Page   The Internet Communications Engine (Ice) is a modern alternative to object middleware such as CORBA\\x99 or COM/DCOM/COM+. Ice is easy to learn, yet provides a powerful network infrastructure for demanding technical applications. Ice shines where technologies such as SOAP or XML-RPC are too slow, or do not provide sufficient scalability or security\\n\\n\\n\\n\\n\\n\\nurlLink Beer Church Home Page\\n\\n\\n\\n\\n\\n\\nurlLink Build Web apps with Maypole   Simon Cozens turns a love of beer into a Perl application server -- going from a simple front end to database servers, and developing into a social-network Web application. He begins, however, with the beer.\\n\\n\\n\\n\\n\\n\\nurlLink Maypole / Apache::MVC\\n\\n\\n\\n\\n\\n\\nurlLink The Perl CD Bookshelf\\n\\n\\n\\n\\n\\n\\nurlLink ONLamp.com: Network Your Shell Scripts with Netpipes [May. 27, 2004]\\n\\n\\n\\n\\n\\n\\nurlLink OnlineOrganizing.com -- Get Work Organizing Tips At Our Clickable Office\\n\\n\\n\\n\\n\\n\\nurlLink Visio Download Sites\\n\\n\\n\\n\\n\\n\\nurlLink IBM MD04: Design rules, naming convention and minimum symbol set for MQSe\\n\\n\\n\\n\\n\\n\\nurlLink IBM MD04: Design rules, naming convention and minimum symbol set for MQSe\\n\\n\\n\\n\\n\\n\\nurlLink jjg.net: a visual vocabulary\\n\\n\\n\\n\\n\\n\\nurlLink studio id | resources\\n\\n\\n\\n\\n\\n\\nurlLink Stencil Download\\n\\n\\n\\n\\n\\n\\nurlLink LookSmart\\n\\n\\n\\n\\n\\n\\nurlLink Telecommunications: Building a better billing model: real-time service controls are key for wireless service providers - Wireless\\n\\n\\n\\n\\n\\n\\nurlLink Telecom Asia - Building a new billing model\\n\\n\\n\\n\\n\\n\\nurlLink Sillyness Spelled Wrong Intentionally ? Faking it: WordPress as CMS   In this tutorial we will cover the steps necessary to use WordPress as a CMS.  This tutorial will cover the basic concepts and steps to get something up and running as quickly as possible.\\n\\n\\n\\n\\n\\n\\nurlLink Crackbaby.com - For The Twisted Little Kid in You!   Ok, this story is for you geeks out there that actually know what you are doing. It requires editing of your registry and should not be done by anyone who doesn\\'t know what they are doing.  This will effectively neuter IE on your system and divert all shell calls to IE to your alternative browser. Read on for details...\\n\\n\\n\\n\\n\\n\\nurlLink IronPython - A fast Python implementation for .NET and Mono    IronPython is a new Python implementation targeting the   .NET   and   Mono   platforms. It is...    Fast  - IronPython-0.6 is up to 1.7x faster than Python-2.3 on the standard pystone benchmark. An early performance report is are contained in this paper for PyCon 2004.    Integrated with the Common Language Runtime  - IronPython code can easily use CLR libraries and Python classes can extend CLR classes.    Fully dynamic  - IronPython supports an interactive interpreter and transparent on-the-fly compilation of source files just like standard Python.    Optionally static  - IronPython also supports static compilation of Python code to produce static executables (.exe\\'s) that can be run directly or static libraries (.dll\\'s) that can be called from other CLR languages including C#, VB, managed C++ and many more.    Managed and verifiable  - IronPython generates verifiable assemblies with no dependencies on native libraries that can run in environments which require verifiable managed code.    Not finished  - IronPython is currently at a pre-alpha stage suitable for experimentation but not for serious development work. \"\\n\\n\\n\\n\\n\\n\\nurlLink s9y PHP weblog software - Serendipity : \"Serendipity is a weblog/blog system, implemented with PHP. It is standards compliant, feature rich and open source (BSD License).\"\\n\\n\\n\\n\\n\\n\\nurlLink ONLamp.com: Getting Started with PHP\\'s  HTML_QuickForm      urlLink Getting Started with PHP\\'s HTML_QuickForm  by Keith Edmunds -- Web programming can be dull the second time you have to validate client parameters or render data to form elements. Fortunately, PHP\\'s HTML_QuickForm class simplifies the creation, processing, and validation of client parameters. Keith Edmunds explains how to start using QuickForm.\\n\\n\\n\\n\\n\\n\\nurlLink Science for Monks- Welcome!   In 2000 His Holiness the Dalai Lama, initiated the Science Education Project to introduce science education to monastic religious scholars in India.\\n\\n\\n\\n\\n\\n\\nurlLink Prog - Page Rank Search Engine\\n\\n\\n\\n\\n\\n\\nurlLink EPIC - Eclipse Perl Integration   EPIC is a collection of Perl plug ins for Eclipse. Currently the following plugins are under development:   - Perl Editor Plugin - Perl Debugger Plugin - RegExp Plugin\\n\\n\\n\\n\\n\\n\\nurlLink PL/FLOW Workflow Engine for Oracle   PL/FLOW is the name of a workflow engine written in PL/SQL. Why use it? It\\'s open source, it\\'s fast, and it can run in a replicated environment with multiple master databases. The naming, process definition and API are inspired by interface 1 and 2 as published by the WfMC. The ERD design was made using Designer 9i. PL/FLOW is lightweight because Oracle database functionality is used for storage, indexing and logic. Workflow transactions typically take 5-10 milliseconds to complete.\\n\\n\\n\\n\\n\\n\\nurlLink SS7 Tutorials,SS7 Resouces,SS7 Products (Inc C7) Etc.\\n\\n\\n\\n\\n\\n\\nurlLink SDBA Revolution Instant Messaging Application Server - Welcome   SDBA Revolution Instant Messaging Application Server is an open-source server written in Perl to provide an architecture to simplify and streamline the process of writing applications that run on an Instant Messaging network.   It can: Just like web server scripting, especially PHP or mod_perl  Allows scripting of IM responses/pages  Uses code tags   to allow interspersed code/plain text  Drop-in pages, real-time changes without restarting server  Supports multiple applications per server  Code can be written using all the powers of Perl  Wide range of pre-set variables available   Tracks user sessions, with configurable timeout  Allows session variables across responses/pages  Easily expandable with useful functions  Supports any number of access lists for basic security  VERY easy to write sohpisticated, useful, and intuitive apps  Currently supports AIM, MSN, ICQ, YIM, and Jabber  Supports MSN Mobile clients - the only bot that does!  Even the most clueless convert will write useful apps quickly\\n\\n\\n\\n\\n\\n\\nurlLink CMSimple Content Management - CMSimple : \"CMSimple is a simple content management system for smart maintainance of small commercial or private sites. It is simple - small - smart!\"\\n\\n\\n\\n\\n\\n\\nurlLink RSS2WAP.com: RSS conversion for WAP (RSS2WML, RSS WAP or RSS-to-WAP) + WAP directory,WAP links : \"This site is meant for online conversions of RSS feeds into WML for access on WAP devices (RSS2WML, RSS WAP or RSS-to-WAP). It\\'s free and you can make a link on your web site. Do you have WAP enabled mobile phone? Why not use it to read news or weblogs online? There are not many sites which offer WML pages but you\\'ll be surprised how many sites offer RSS feeds.\"\\n\\n\\n\\n\\n\\n\\nurlLink Understand Enterprise Service Bus scenarios and solutions in Service-Oriented Architecture, Part 3 : \"In Part 3 of this series on scenarios and solutions for implementing an Enterprise Service Bus, the author examines possible solutions for the various scenarios outlined in Part 2. The ideas on the role of the Bus as explained in Part 1 provide the foundation for the scenarios. \"\\n\\n\\n\\n\\n\\n\\nurlLink Understand Enterprise Service Bus scenarios and solutions in Service-Oriented Architecture, Part 2 : \"In Part 2 of this series on the Enterprise Service Bus (EBS), the author describes and analyzes some commonly observed scenarios in which ESBs and other Service-Oriented Architecture (SOA) solutions are implemented. \"\\n\\n\\n\\n\\n\\n\\nurlLink Understand Enterprise Service Bus scenarios and solutions in Service-Oriented Architecture, Part 1   This article identifies a set of minimum capabilities that fulfill the most basic needs for an Enterprise Service Bus (ESB) consistent with the principles of Service-Oriented Architecture (SOA). Identifying these minimum capabilities allows you to identify which existing technologies to use to implement an ESB to support an SOA. By considering how the requirements of a specific situation define the need for additional capabilities, you can then choose the most appropriate implementation technology for that situation.\\n\\n\\n\\n\\n\\n\\nurlLink PBS | I, Cringely . Archived Column\\n\\n\\n\\n\\n\\ni have no idea what to put here\\n\\n\\n\\n\\n\\nI really am in the wrong line of work. Hospitality. I couldn\\'t be any less hospitable if I tried. Don\\'t get me wrong, I\\'m very polite to customers. I always smile and say Hello! But it only takes the SLIGHTEST wrong doing on their part and I quickly turn. People really do leave their manners at the door. I refuse to be polite to people who wont return the favour. Here is a wee list of things that really piss me off. Little things that have started to fester and turned into the biggest deals on earth. I know they may seem LITTLE, but you try having it happen to you day and and day out. YOU JUST TRY IT. And yes I know i could quit but I dont wanna.  1. Thowing money on the counter. You may as well be throwing money at your feet and telling me to beg for it. FUCK YOU. 2. Not saying please or thank you. It\\'s pretty basic. 3. Having me say Hello! and the customer just saying GIVE ME THIS and not even LOOKING at you. Fuck that makes me mad. 4.Whinging about the cost. You knew how much it was when you were ordering it.  5. Getting mad at ME because they have to wait in a line. Dont wait in the LINE THEN. 6. Talking on your cellphone while a big line forms behind you and putting your hand IN MY FACE TO STOP ME ASKING WHAT YOU WANT. This happens ALL THE TIME. So rude.  7. People leaving their tables in a filthy mess of sugar packets and food scraps. Like they had a crazy feeding frenzy. Disgusting pigs.  Today a lady asked my co-worker to please put butter on her muffin for her, which she was taking away with her. She stood there and directed where abouts and how much butter to put in which places. Why must people degrade us this way? What the fuck is wrong with your hands that you cant do it? the only time somebody has ever asked me to do that was when they had two CASTS on their arms. Am I the only one who thinks this is DISGUSTING? I was so mad I could have split. I could write for days and days about how mad my job makes me. But just when you think you cant stand seeing another UNGREATFUL THANKLESS ASSHOLE, somebody lovely comes in and is actually NICE to you and you feel all better.\\n\\n\\n\\n\\n\\nBonjour.  Today I saw an event which made me cry for the state of the world and rejoice the goodness of others all in the space of a few seconds. Which is a common theme i guess...good and evil etc...etc. Yawn. Anyway here\\'s the story.  So there I was, standing at work with no customers to tend to, staring out into the early night sky and the brightly lit street. Watching people walking home from work, clad in woolen jackets, briskly rubbing woolen hands tog--anyways, SUDDENLY these three \"kids\" come along, one of them in a wheelchair pretending to be mentally disabled with the other two kicking the chair yelling RETARD!! That probably sounds like it was funny, but it really wasn\\'t. I dont mean that in an overly sensitive girl way, either. It was just depressing. Like, what the hell?  Then! Suddenly a grown man RACES up, grabbing one of the kids by the scruff of the neck and cries \"DONT YOU EVER DO THAT AGAIN YOU INSENSITIVE FUCKING LITTLE PRICK\" and the kids REELED back in TERROR. It was like an inaudable cheer went up all over wellington! Finally! Somebody had enough guts to say HEY!! WHAT YOU ARE DOING IS HORRIBLY UNACCEPTABLE!! It was a great moment indeed. I felt slightly elated. I felt as though this would be how I would deal to situations from now on. No more sitting back and watching it happen, no sir. Then I realised that I wouldn\\'t change at all and I felt ashamed and depressed for a moment, then I stopped thinking about it altogether because somebody came into the store and started DEMANDING a drink. No need for manners! I DONT HAVE FEELINGS I\"M JUST A ROBOT SLAVE. Oops, always off on the tangents.  Bye!\\n\\n\\n\\n\\n\\nThis is a \"rant\" about something that sometimes really pisses me off. It\\'s not really a big deal, and if this is all i have to worry about then i should be glad BLAH BLAH WHATEVER, but I still feel I am entitled to be mad about it and VENT MY RAGE in text, here it is.  My boyfriend, lets call him \"Tim\", never EVER does anything to help me keep the house tidy. I don\\'t mean insanely tidy, i just mean a low-level kind of tidy. Comfortable tidy. I have a lot of stuff, so things get untidy easily. Tim does not do one single thing to help my cause. It was okay before, when he was earning all the money and I just sat at home eating chocolates and watching Dr Phil, but now i give him most of the money I earn slaving away at my job six days a week. I contribute! But the burden of having to look after EVERYTHING is still firmly rested on my shoulders.   Tim looks after the bills and makes sure everything gets paid, etc, etc. But that only takes a few minutes because he does it all ONLINE LIKE A BIG NERD. I spend at least two hours a day doing washing and making beds and so on and so on and so on. Sometimes I say...Hey tim, maybe you could do the dishes tonight, i\\'m really tired. Sure! he says, \"I\\'ll do it tomorrow, before i go to work.\" .....what?  Here\\'s a little example of a typical situation that pisses me off so much i feel like taking a knife and slashing my own face to bits so i can go to hospital and not have to do any housework f--anyway, here\\'s an example. Tim likes to take his clothes off before he goes to bed in the LOUNGE. and leave them on the CHAIR and on the floor. Apparently its warmer in there. I dont need to go into the ten million reasons why this is a completely idiotic reason, as I feel it is fairly apprent. Next day, Oh, my SHIRTS ALL CRUMPLED, Liz can you IRON it for me? Cue sad face. His, not mine, mines more of a rage face. Liz if you get time before work and inbetween cleaning my underpants can you please wash all my shirts for me? Ta.  The fact that I have to tidy things away seems to be the reasoning behind the fact Tim can never FIND anything. You tidyed it away! Where have you put my \"......\". He totally flips out. This morning i was woken up at 7:30 AM on the day of my ONLY sleep in, because \"LIZ I HAVE LOST MY WALLET\" so i get up. Stumble around the house and say HEY\\n\\n\\n\\n\\n\\nOpen letter to: My Future Internet Friend  Hi! How are you? I am fine. I am writing to let you know that, no matter what you think now, one day we will be the best of internet friends. Regardless of the fact that right now you consider me a total psycho, one day this will appear endearing and you will look upon it in a loving, knowing light. Don\\'t worry, I love you too.  So, Dearest, Future Internet Friend (obviously I\\'m not talking about you Jeremy, or i would say Fickilist, time-hogging current internet friend who i am currently seeking replacement for) I hope you took this time to read this and consider my application. I\\'m a really GREAT internet friend, and promise not to ditch you as soon as something better comes along, unlike some people. I\\'m looking at you, JJB.  Bye!\\n\\n\\n\\n\\n\\nHello again. And a special hello to my most dedicated reader, Jeremy Burtenshaw. Hope everything is going well with the new template hunt.   Today was completely reproachable in every respect. Thousands of greedy, impatient and most noticably obsese and ugly customers were in a continuous state of trying to jam themselves through the narrow entrance of the shop. While the three of us coward behind the counter desperatly trying to throw together drinks and food orders while they stood sullen and homourless, alternating from staring at their watches to staring at us with their evil, unforgiving eyes. After lurching around trying to please everyone for around five solid hours we were then given the luxury of closing the shop and spending two hours washing their cheap waxy lipstick from the rims of the cups. Oh, thanks for stomping your stale cookie into the carpet, that really made my fucking day.\\n\\n\\n\\n\\n\\nWell dedicated readers, it\\'s been a while since my last post (as you must have noticed! dont worry!! i didnt commit suicide! even though i threatened to in my quit message, soz :/) but i\\'m back now and ready to regale and entertain you all!\\n\\n\\n\\n\\n\\nHi!  Whenever I\\'m feeling lame enough to keep a \"diary\" or write stuff places in say...a BLOG, I cant ever write anything when something important is happening. My diarys always die out when something great has happened or something depressing has happened, then I feel I cant pick them up again without some kind of explaination to myself about it. In written form. Which I really struggle with. So I thought I should break that now so I will write something here again.  On the weekend I got engaged! To be MARRIED! OMG.  K that\\'s all.\\n\\n\\n\\n\\n\\nPreparing my Libraries orientation for graduate computer science students, and found this fascinating article on  urlLink  ACM Queue  vol. 1, no. 4 - June 2003 - \"How Much Storage is Enough?\"  : \"In 1999, the world produced about 1.5 exabytes of storable content (1018 bytes). This is 1.5 billion gigabytes, and is equivalent to about 250 megabytes for every man, woman, and child on earth. Printed documents of all kinds make up only .003 percent of the total. Magnetic storage is by far the largest medium for storing information and is the most rapidly growing, with shipped hard-drive capacity doubling every year. Magnetic storage is rapidly becoming the universal medium for information storage...Although the social impact of the Web has been phenomenal, about 500 times as much e-mail is being produced per year than Web pages. It appears that about 610 billion e-mails are sent per year, compared with 2.1 billion static Web pages. Even the yearly flow of Usenet news is more than three times the stock of Web pages. As Andrew Odlyzko puts it,  \"Communication, not content, is the killer app.\"  \"\\n\\n'"

In [46]:
nb_predict(blogposts_model, post)

'male'

Make training and testing sets.

In [54]:
try:
    os.mkdir('./blogs_train')
except:
    pass
try:
    os.mkdir('./blogs_test')
except:
    pass

In [58]:
data_dir = './data/data/'
train_dir = './blogs_train'
test_dir = './blogs_test'
for file in os.listdir(data_dir):
    if file[0] == '4' or file[0] == '8':
        shutil.copy2(data_dir + file, train_dir)
    elif file[0] == '6' or file[0] == '7':
        shutil.copy2(data_dir + file, test_dir)